In [1]:
import math
#global variable
doNormalization=True
useGpu=False
runOnG814=False
isTest=True
modelPath='/home/wangyuchen/wholeNet_300epoch_50perEpoch.pt'
maxMatrixIndex=250
runRelationLSTM=False
runObjectRelationNet=False
runSeq2SeqRelationModel=True
runLSTM=False

maxRelationsNumberGlobal=20 # the maximum nubmer of relation in the "relation within the given range" version 

if useGpu:
    import os
    os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [54]:
def fromAllToStr(*args):
    returnedStr=str()
    for eachItem in args:
        returnedStr=returnedStr+str(eachItem)
    return returnedStr

In [3]:
import logging
logging.basicConfig(filename='logging.txt',level=logging.DEBUG, format='%(asctime)s -%(lineno)d - %(funcName)s - %(levelname)s - %(message)s',)
console = logging.StreamHandler()
console.setLevel(logging.DEBUG)
formatter = logging.Formatter('%(asctime)s -%(lineno)d - %(funcName)s - %(levelname)s - %(message)s')
console.setFormatter(formatter)
logging.getLogger('').addHandler(console)

In [4]:
#generate a tuple in which each element is the index of a vehicle
#the tuple is used to get the property of each vehicle from the left side of data from the dataset function
listForEachVehicle=[]
for i in range(maxMatrixIndex):
    listForEachVehicle.append(i*(maxMatrixIndex-1))
tupleForEachVehicle=tuple(listForEachVehicle)

In [5]:
import numpy as np
import torch

def getValueByLable(lableList,valueList):
    """
    For instance, given a lable list ['Local_X','Local_Y'] and a value list [2.0, 24.0, 437.0, 1118846981300.0, 16.254, 
    79.349, 6451167.199, 1873312.382, 14.5, 4.9, 2.0, 39.14, -5.73, 2.0, 0.0, 13.0, 0.0, 0.0] which values sorted by the 
    order of allLableList below, the function return a value Dict {'Local_X':16.254, 'Local_Y':79.349}
    Args:
        lableList: the list of lables you've required, such as['Vehicle_ID', 'Total_Frames','Global_Time']
        valueList: the list contains all legally value, sorted by:['Vehicle_ID', 'Frame_ID','Total_Frames','Global_Time','Local_X','Local_Y','Global_X','Global_Y',\
                      'v_Length','v_Width','v_Class','v_Vel','v_Acc','Lane_ID','Preceding','Following','Space_Headway',\
                      'Time_Headway']
    Returns: 
        value dict of the input lables
    For instance, given a lable list ['Local_X','Local_Y'] and a value list [2.0, 24.0, 437.0, 1118846981300.0, 16.254, 
    79.349, 6451167.199, 1873312.382, 14.5, 4.9, 2.0, 39.14, -5.73, 2.0, 0.0, 13.0, 0.0, 0.0] which values sorted by the 
    order of allLableList above, the function return a value List [16.254, 79.349]

    """
    allLableList=['Vehicle_ID', 'Frame_ID','Total_Frames','Global_Time','Local_X','Local_Y','Global_X','Global_Y',\
                  'v_Length','v_Width','v_Class','v_Vel','v_Acc','Lane_ID','Preceding','Following','Space_Headway',\
                  'Time_Headway']
    valueDictReturn={}
    for lableItem in lableList:
        valueDictReturn[lableItem]=valueList[allLableList.index(lableItem)]
    return valueDictReturn

def rearrangeDataByGlobalTime(allValueLists):
    '''
    Args:
        allValueLists: all values have been read from a txt file which have already been converted to a list
    Returns:
        dict have been arranged by global time. One single global time generally contains several value lists.
    '''
    valueDict={}
    for valueList in allValueLists:
        dictKey=getValueByLable(['Global_Time'],valueList)['Global_Time']
        if dictKey in valueDict:
            # if dictKey already there, then add valueList to the list of the key
            valueDict[dictKey].append(valueList)
        else:
            #else, create a list and append valueList on it
            valueDict[dictKey]=[valueList]
    return valueDict

In [6]:
import copy

def readFirstFrame(matrixIndexAndVehicleIDRecordDictParam, valueLists):
    """
    To generate the first set of tensors from the first frame
    Args:
        matrixIndexAndVehicleIDRecordDictParam: just as its name
        valueLists: a list consists of all valuelist at one time
    Returns:
        several tensors arranged by: positionTensor, speedTensor, accTensor, angleTensor,newVehicleList(type:list)
    
    """
    maxMatrixIndex=matrixIndexAndVehicleIDRecordDictParam.keys().__len__()-1
    #tensors initialize
    positionTensor=torch.zeros(2,maxMatrixIndex)
    speedTensor=torch.zeros(1,maxMatrixIndex)
    accTensor=torch.zeros(1,maxMatrixIndex)
    angleTensor=torch.zeros(1,maxMatrixIndex)
    newVehicleIDList=[]
    curMatrixIndex=0
    matrixIndexAndVehicleIDRecordDictParam['time']=getValueByLable(["Global_Time"],valueLists[0])['Global_Time']
    #fill out all tensors
    for eachValueList in valueLists:
        #get values from eachValueList, generate dict
        returnedEachValueDict=getValueByLable(['Vehicle_ID','Local_X','Local_Y','v_Vel','v_Acc'],eachValueList)
        #assign to the curMatrixIndex-th row of corresponding tensor
        #angle Tensor assignment is not neeed for the initial value of each element in it is already zero
        positionTensor[:,curMatrixIndex]=torch.tensor((returnedEachValueDict['Local_X'],returnedEachValueDict['Local_Y']))
        speedTensor[:,curMatrixIndex]=torch.tensor((returnedEachValueDict['v_Vel']))
        accTensor[:,curMatrixIndex]=torch.tensor((returnedEachValueDict['v_Acc']))
        #then handle the record matrix
        matrixIndexAndVehicleIDRecordDictParam[curMatrixIndex]['Vehicle_ID']=returnedEachValueDict['Vehicle_ID']
        matrixIndexAndVehicleIDRecordDictParam[curMatrixIndex]['refresh']=0
        newVehicleIDList.append(copy.deepcopy(returnedEachValueDict['Vehicle_ID']))
        curMatrixIndex=curMatrixIndex+1
    return positionTensor,speedTensor,accTensor,angleTensor,newVehicleIDList

In [7]:
def findMatrixIndexByVehicleID(matrixIndexAndVehicleIDRecordDictParam, vehicle_ID):
    for i in range(0, len(matrixIndexAndVehicleIDRecordDictParam)-1):
        if matrixIndexAndVehicleIDRecordDictParam[i]['Vehicle_ID']==vehicle_ID:
            return i
    return -1

def findEmptyMatrixIndex(matrixIndexAndVehicleIDRecordDictParam):
    for i in range(0, len(matrixIndexAndVehicleIDRecordDictParam)-1):
        if matrixIndexAndVehicleIDRecordDictParam[i]['Vehicle_ID']==-1:
            #Vehicle_ID=-1 when there is no existed vehicle ID bounding to the index
            return i
    raise Exception("NO EMPTY ELEMENT IN MATRIX")

def readGeneralFrame(matrixIndexAndVehicleIDRecordDictParam, valueLists, prePositionTensor):
    """
    To generate the first set of tensors from the general frame that have a preceding one.
    In this version, we ignore the new vehicle appeared among a serial of frame.
    Args:
        matrixIndexAndVehicleIDRecordDictParam: just as its name
        valueLists: a list consists of all valuelist at one time
        prePositionTensor: positionTensor from the preceding frame, which is used to calculate angle tensor
    Returns:
        everal tensors arranged by: positionTensor, speedTensor, accTensor, angleTensor,newVehicleList(type:list),
        vanishedVehicleList(type:list)
    
    """
    #tensors initialize
    maxMatrixIndex=matrixIndexAndVehicleIDRecordDictParam.keys().__len__()-1
    positionTensor=torch.zeros(2,maxMatrixIndex)
    speedTensor=torch.zeros(1,maxMatrixIndex)
    accTensor=torch.zeros(1,maxMatrixIndex)
    angleTensor=torch.zeros(1,maxMatrixIndex)
    newVehicleIDList=[]
    vanishedVehicleList=[]
    curMatrixIndex=0
    matrixIndexAndVehicleIDRecordDictParam['time']=getValueByLable(["Global_Time"],valueLists[0])['Global_Time']
    #fill out all tensors
    for eachValueList in valueLists:
        #get values from eachValueList, generate dict
        returnedEachValueDict=getValueByLable(['Vehicle_ID','Local_X','Local_Y','v_Vel','v_Acc'],eachValueList)
        indexOfVehicle=findMatrixIndexByVehicleID(matrixIndexAndVehicleIDRecordDictParam,returnedEachValueDict['Vehicle_ID'])
        if indexOfVehicle!=-1:
        #if index exist then the vehicle already existed in the preceded frame
            matrixIndexAndVehicleIDRecordDictParam[indexOfVehicle]['refresh']=1
            curMatrixIndex=indexOfVehicle
            #assign to the curMatrixIndex-th row of corresponding tensor
            positionTensor[:,curMatrixIndex]=torch.tensor((returnedEachValueDict['Local_X'],returnedEachValueDict['Local_Y']))
            speedTensor[:,curMatrixIndex]=torch.tensor((returnedEachValueDict['v_Vel']))
            accTensor[:,curMatrixIndex]=torch.tensor((returnedEachValueDict['v_Acc']))
            angleTensor[:,curMatrixIndex]=math.atan2(positionTensor[0,curMatrixIndex]-\
                                                     prePositionTensor[0,curMatrixIndex],\
                                                    positionTensor[1,curMatrixIndex]-prePositionTensor[1,curMatrixIndex])
        else:
            pass #ignore new vehicleID
        #a new vehicle ID
#             newVehicleIDList.append(copy.deepcopy(returnedEachValueDict['Vehicle_ID']))
#             curMatrixIndex=findEmptyMatrixIndex(matrixIndexAndVehicleIDRecordDictParam)
#             matrixIndexAndVehicleIDRecordDictParam[curMatrixIndex]['Vehicle_ID']=copy.deepcopy(returnedEachValueDict['Vehicle_ID'])
#             matrixIndexAndVehicleIDRecordDictParam[curMatrixIndex]['refresh']=1
#             #assign to the curMatrixIndex-th row of corresponding tensor
#             positionTensor[:,curMatrixIndex]=torch.tensor((returnedEachValueDict['Local_X'],returnedEachValueDict['Local_Y']))
#             speedTensor[:,curMatrixIndex]=torch.tensor((returnedEachValueDict['v_Vel']))
#             accTensor[:,curMatrixIndex]=torch.tensor((returnedEachValueDict['v_Acc']))
#             angleTensor[:,curMatrixIndex]=math.atan2(positionTensor[0,curMatrixIndex]-\
#                                                      prePositionTensor[0,curMatrixIndex],\
#                                                     positionTensor[1,curMatrixIndex]-prePositionTensor[1,curMatrixIndex])
    for i in range(0,maxMatrixIndex):
    #find vanished vehicle and remove from dict
        if matrixIndexAndVehicleIDRecordDictParam[i]['refresh']==0:
            #if refresh=0 then the corresponding vehicle ID was not found in this frame
            vanishedVehicleList.append(copy.deepcopy(matrixIndexAndVehicleIDRecordDictParam[i]['Vehicle_ID']))
            matrixIndexAndVehicleIDRecordDictParam[i]['refresh']=-1
            matrixIndexAndVehicleIDRecordDictParam[i]['Vehicle_ID']=-1
    
    for i in range(0,maxMatrixIndex):
    #set all refrshed which equivalent to 1 to 0 to prepare for the next frame
        if matrixIndexAndVehicleIDRecordDictParam[i]['refresh']==1:
                #if refresh=0 then the corresponding vehicle ID was not found in this frame
                matrixIndexAndVehicleIDRecordDictParam[i]['refresh']=0

    return positionTensor,speedTensor,accTensor,angleTensor,newVehicleIDList,vanishedVehicleList


In [8]:
import torch.utils.data as utils
import torch.nn.functional as F
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.nn.parameter import Parameter
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
if not runOnG814:
    %matplotlib inline 

2020-08-02 22:24:36,054 -276 - wrapper - DEBUG - $HOME=/home/wangyuchen
2020-08-02 22:24:36,056 -276 - wrapper - DEBUG - CONFIGDIR=/home/wangyuchen/.config/matplotlib
2020-08-02 22:24:36,056 -276 - wrapper - DEBUG - matplotlib data path: /home/wangyuchen/.conda/envs/pytorch/lib/python3.6/site-packages/matplotlib/mpl-data
2020-08-02 22:24:36,059 -1007 - rc_params_from_file - DEBUG - loaded rc file /home/wangyuchen/.conda/envs/pytorch/lib/python3.6/site-packages/matplotlib/mpl-data/matplotlibrc
2020-08-02 22:24:36,062 -1644 - <module> - DEBUG - matplotlib version 3.1.3
2020-08-02 22:24:36,062 -1645 - <module> - DEBUG - interactive is False
2020-08-02 22:24:36,063 -1646 - <module> - DEBUG - platform is linux
2020-08-02 22:24:36,063 -1647 - <module> - DEBUG - loaded modules: ['builtins', 'sys', '_frozen_importlib', '_imp', '_warnings', '_thread', '_weakref', '_frozen_importlib_external', '_io', 'marshal', 'posix', 'zipimport', 'encodings', 'codecs', '_codecs', 'encodings.aliases', 'encodin

2020-08-02 22:24:36,109 -276 - wrapper - DEBUG - CACHEDIR=/home/wangyuchen/.cache/matplotlib
2020-08-02 22:24:36,112 -1360 - <module> - DEBUG - Using fontManager instance from /home/wangyuchen/.cache/matplotlib/fontlist-v310.json
2020-08-02 22:24:36,223 -225 - switch_backend - DEBUG - Loaded backend module://ipykernel.pylab.backend_inline version unknown.
2020-08-02 22:24:36,228 -225 - switch_backend - DEBUG - Loaded backend module://ipykernel.pylab.backend_inline version unknown.
2020-08-02 22:24:36,230 -225 - switch_backend - DEBUG - Loaded backend module://ipykernel.pylab.backend_inline version unknown.


In [9]:
def fromDirGenerateDict(trajectoryDir):
    """
    Returns:
        valueDict: the key is global time, and the value of each key contain SEVERAL LIST of properties, 
                   each list consist of all property of a single vehicle at one time.
    """
    trajectoryDataFile=open(trajectoryDir)
    count=0
    allLineList=[]
    count=0
    for count,line in enumerate(trajectoryDataFile):
        #read a single line, remove space and enter
        lineList=line.split(' ')
        try:
            while True:
                lineList.remove('')
        except:
            try:
                lineList.remove('\n')
            except:
                pass
            pass
        for i in range(0,lineList.__len__()):
            # convert string to float
            lineList[i]=float(lineList[i])
        allLineList.append(lineList)
    valueDict=rearrangeDataByGlobalTime(allLineList)
    return valueDict

In [10]:
def getMaxAndMinValueFromValueDict(valueDict,lableList):
    """
    Args:
        valueDict: each key in dict is global time, the value of each key is a list of all value at one time
        lableList: lables from which you want to get the max and min value. the type of each value in the list 
                    is str.
    Returns:
        a dict, which has keys keys from the input lable list and the value of each key is a dict which formed
        as 'max':value, 'min':value
    """
    maxAndMinDict={}
    keys=list(valueDict.keys())
    for lable in lableList:
        max=0
        min=0 #speed,  positon are all from 0 to max, accelerate from - to +
        for eachKey in keys:
            valueLists=valueDict[eachKey]
            for valueList in valueLists:
                value=getValueByLable([lable],valueList)[lable]
                if value>max:
                    max=value
                if value<min:
                    min=value
        maxAndMinDict[lable]={'max':max,'min':min}
    return maxAndMinDict

In [11]:
# #test function of finding the max and min value: block 1, get valueDict for saving time from file reaidng
# valueDict=fromDirGenerateDict(1)


In [12]:
#test function of finding the max and min value: block 1, get valueDict for saving time from file reaidng
# getMaxAndMinValueFromValueDict(valueDict,['Local_X','Local_Y','v_Acc','v_Vel'])

In [13]:
# valueDict=fromDirGenerateDict(1)
# theKey=list(valueDict.keys())[1000]

In [14]:
def visualizeTensorData(xTensor,yTensor, maxLength=2500,maxWidth=100,blocksize=10,normalizationDict=False):
    """
    visualize a frame on an white image
    Args:
        valueVisualize: a list of values, each item in the list can be obtained by function 
        getValueByLable
    Returns:
        the image of the input frame
    """
    image=np.ones((maxLength,maxWidth,3),dtype=np.int8)
    image=image*255
#     figure=plt.figure(figsize=(10,50))
#     axe=figure.add_subplot(1,1,1)
    xLength=xTensor.shape[0] #the length of y is equivalent to x's
#     print('length:',xLength)
#     print('xTensor.shape',xTensor.shape)
    if doNormalization&(normalizationDict is not False):
        originalXTensor=torch.zeros(xLength)
        originalYTensor=torch.zeros(xLength) #originalX and Y tensor share the same length
        originalXTensor=torch.add(\
                                  torch.mul(xTensor,normalizationDict['positionXMax']-normalizationDict['positionXMin']),\
                                  torch.add(originalXTensor,normalizationDict['positionXMin'])
                                 )
        originalYTensor=torch.add(\
                                  torch.mul(yTensor,normalizationDict['positionYMax']-normalizationDict['positionYMin']),\
                                  torch.add(originalYTensor,normalizationDict['positionYMin'])
                                 )
        for i in range(xLength):
            x=int(originalXTensor[i])
            y=int(originalYTensor[i])
            colorR=int((i*17+29)%255)
            colorG=int((i*9++93)%255)
            colorB=int((i*13+111)%255)
            cv2.circle(image,(x,y),int(blocksize/2),(colorB,colorG,colorR),-1) #
    #     axe.imshow(image)
        return image
        
    
    
    for i in range(xLength):
        x=int(xTensor[i])
        y=int(yTensor[i])
        colorR=int((i*17+29)%255)
        colorG=int((i*9++93)%255)
        colorB=int((i*13+111)%255)
        cv2.circle(image,(x,y),int(blocksize/2),(colorB,colorG,colorR),-1) #
#     axe.imshow(image)
    return image

In [15]:
import cv2
import matplotlib as mpl
import matplotlib.pyplot as plt
if not runOnG814:
    %matplotlib inline
from IPython import display
def visualizeData(valueVisualize, maxLength=1000,maxWidth=100,blocksize=10):
    """
    visualize a frame on an white image
    Args:
        valueVisualize: a list of values, each item in the list can be obtained by function 
        getValueByLable
    Returns:
        the image of the input frame
    """
    image=np.ones((maxLength,maxWidth,3),dtype=np.int8)
    image=image*255
#     figure=plt.figure(figsize=(10,50))
#     axe=figure.add_subplot(1,1,1)
    
    for item in valueVisualize:
        infoList=getValueByLable(['Vehicle_ID','Local_X','Local_Y'],item)
        vehicleID=infoList['Vehicle_ID']
        x=int(infoList['Local_X'])
        y=int(infoList['Local_Y'])
        colorR=int((vehicleID+100)%255)
        colorG=int((vehicleID+150)%255)
        colorB=int((vehicleID+200)%255)
        cv2.circle(image,(x,y),int(blocksize/2),(colorB,colorG,colorR),-1) #
#     axe.imshow(image)
    return image
# visualizeData(valueDict[theKey])




2020-08-02 22:24:36,352 -225 - switch_backend - DEBUG - Loaded backend module://ipykernel.pylab.backend_inline version unknown.


In [63]:
def discountParameterByExponentialWithDistance(relationTensors, xfactor=1, yfactor=1, w1=1,w2=2,w3=1):
    """
    to calculate a discount parameter matrix from the relations matrix of several vehicl pairs.
    Args:
        relationTensors: a vehicle pair is a "relation" in which two vehilce properties are of the same kind and order,
        the relationTensors consists of many vehicle pairs. An extra dimension should be added to the a single vehicle pair
        tensor if this function only take as a single vehicle pair. Generally, the relationTensors has to dimension:
        the dimension of pairs, the properties of each pair.
        xfactor,yfactor:  the weight of x part and y part
        w1, w2,w3: facor in exponential operation, w1 and w2 relate to x and w3 relate to y
    """
    vehiclePairDimension=relationTensors.shape[1] #the second dimension of relationTensors is the properties of each vehicle pair
    relationsDimension=relationTensors.shape[0]
    computationTensor=torch.zeros((relationsDimension,5)) #save the xDifference and yDifference for further computation
    secondVehiclePropertyStartIndex=int((vehiclePairDimension)/2 )
    logging.debug('secondVehiclePropertyStartIndex'+str(secondVehiclePropertyStartIndex))
    
    discountTensor=torch.zeros(relationsDimension)
    for i in range(relationsDimension):
        x1,y1,x2,y2=relationTensors[i][0],relationTensors[i][1],\
                    relationTensors[i][secondVehiclePropertyStartIndex],relationTensors[i][secondVehiclePropertyStartIndex+1]
        if y2>=y1: #the second vehilce is in front of the first vehicle
            yDifference=y2-y1
            wy=w1
        else: #the second vehicle is after the fist vehicle
            yDifference=y1-y2
            wy=w2
        xDifference=abs(x1-x2)
        wx=w3
        computationTensor[i][0]=xDifference
        computationTensor[i][1]=yDifference
        computationTensor[i][2]=wx
        computationTensor[i][3]=wy
        if (x1==0 and y1==0)or(x2==0 and y2==0):
            computationTensor[i][4]=0
        else:
            computationTensor[i][4]=1
#         discountTensor[i]=(xfactor/math.exp(wx*(xDifference)))*(yfactor/math.exp(wx*(yDifference)))
    logging.debug(fromAllToStr('computationTensor:\n',computationTensor))
    discountTensor=torch.mul(torch.mul((xfactor/torch.exp(torch.mul(computationTensor[:,0],computationTensor[:,2]))),\
                             (yfactor/torch.exp(torch.mul(computationTensor[:,1],computationTensor[:,3])))),
                             computationTensor[:,4])
    return discountTensor
    

In [81]:
#input: property tensors, target vehicle ID
#output: relations, distances between target vehicle and other vehicles which inside the range
def relationCalculateWithRange(propertyTensors, distanceRange, targetVehicleId, maxRelationsNumber=maxRelationsNumberGlobal):
    """
    NOTICE:THE PROPERTIES AND DISTANCE RANGE SHOULD BOTH BE NORMALIZED OR UNNORMALIZED!!!
    Args:
        propertyTensors:property tensors of all vehicles, at position 0 and 1 are the x and y positon of the 
        corresponding vehicle
        distanceRange: the distance on which we decide to take other vehicles into account
        targetVehicleId: the center vehicle which we are going to calculate
    """
    propertyTensorsCopy=copy.deepcopy(propertyTensors)
    propertiesDimension=propertyTensorsCopy.shape[0]
    reservedIndexes=[]
    reservedIndexDistanceDict={}
    for i in range(maxMatrixIndex):
        reservedIndexes.append(i)
    #remove vehicles which position are out-of-range
    for i in range(propertiesDimension):
        #simply remove the out-of-ranged vehicles by comparing y-axis before computing the distance.
        #since most vehicles are out of the given range, this would reduce time cost
        if abs(propertyTensorsCopy[i][1]-propertyTensors[targetVehicleId][1])>distanceRange:
            reservedIndexes.remove(i)
        #to compute if vehicle in the range
        else:
            distance=((propertyTensorsCopy[i][0]-propertyTensors[targetVehicleId][0])**2+\
                (propertyTensorsCopy[i][1]-propertyTensors[targetVehicleId][1])**2)**0.5
            if distance>distanceRange:
                reservedIndexes.remove(i)
            else:
                reservedIndexDistanceDict[i]=distance
    #sort dict by value, not by key
    sortedReservedIndexDistanceDict=sorted(reservedIndexDistanceDict.items(),key=lambda item:(item[1],item[0]))
    #keep the top 'maxRelationsNumber' nearest vehicles in the generated relations
    if(sortedReservedIndexDistanceDict.__len__()>maxRelationsNumber):
        for i in range(maxRelationsNumber,sortedReservedIndexDistanceDict.__len__()):
            reservedIndexes.remove(sortedReservedIndexDistanceDict[i][0])
    #the final properties tensor is:
    finalPropertiesTensor=propertyTensorsCopy[reservedIndexes]
    if(sortedReservedIndexDistanceDict.__len__()<maxRelationsNumber):
        #make sure the length of all relation are equanl to the value of maxRelationNumber
        finalPropertiesTensor=torch.cat((finalPropertiesTensor,\
                                         torch.zeros((maxRelationsNumber-finalPropertiesTensor.shape[0],finalPropertiesTensor.shape[1]))))
    
    logging.debug('targetVehicleId'+str(targetVehicleId))
    logging.debug('reservedIndexes.__len__()'+str(reservedIndexes.__len__()))
    logging.debug('propertyTensors[targetVehicleId].shape[0]'+str(propertyTensors[targetVehicleId].shape[0]))
    expandedTargetVehicleTensor=propertyTensors[targetVehicleId].expand(maxRelationsNumber,propertyTensors[targetVehicleId].shape[0])
    relationTensor=torch.cat((expandedTargetVehicleTensor,finalPropertiesTensor),1)
#     logging.debug('expandedTargetVehicleTensor:'+str(expandedTargetVehicleTensor))
#     logging.debug('relationTensor:'+str(relationTensor))
    return relationTensor
    
    

In [79]:
def computeRelationAndAllTheOtherTensorsWithDistance(inputFrameTensor,theGivenRange,maxRelationsNumber=20):
    '''
        to generate relation tensors, discount parameter tensor with relations and the relation quantity tensor
    of each vehicle
    Args:
        inputFrameTensor: vehicle properties graph, which dimension are (timestep, property, vehicle).
        To illustrate the meaning of dimensions, supposing we have a inputFrameTensor which 
        timestep is 10, all vehicle have 6 properties and there are 250 vehicles, then the dimension of the 
        input tensor are(10, 6, 250)
        theGivenRange: only take vehicle pairs which distance are inside the given range into account.
    Returns:
        relationTensor: the relation tensors of each vehicle pairs in the given range
        discountParameterTensor: the discount tensor of each relation, computed by the distance between vehicle pairs
        
    '''
    
    #circulation for batch list
    logging.debug(fromAllToStr('inputFrameTensor.shape:',inputFrameTensor.shape))
    timeStepSize=inputFrameTensor.shape[0]
    for timeStepCount in range(0,timeStepSize):
        for vehicleId in range(0, inputFrameTensor.shape[2]):
            relationTensor=relationCalculateWithRange(inputFrameTensor[timeStepCount].permute(1,0),
                                                      theGivenRange,vehicleId,maxRelationsNumber=maxRelationsNumber)
            if vehicleId==0:
                relationTensorInOneTimeStep=relationTensor
            elif vehicleId>0:
                relationTensorInOneTimeStep=torch.cat((relationTensorInOneTimeStep,relationTensor),dim=0)
        logging.debug(fromAllToStr('relationTensorInOneTimeStep.shape:\n',relationTensorInOneTimeStep.shape))
        discountParameterTensorInOneTimeStep=discountParameterByExponentialWithDistance(relationTensorInOneTimeStep)
        if timeStepCount==0:
            relationTensorOfAllTimeSteps=relationTensorInOneTimeStep.unsqueeze(0)
            discountParameterTensorofAllTimeSteps=discountParameterTensorInOneTimeStep.unsqueeze(0)
        else:
            relationTensorOfAllTimeSteps=\
            torch.cat((relationTensorOfAllTimeSteps,relationTensorInOneTimeStep.unsqueeze(0)),dim=0)
            discountParameterTensorofAllTimeSteps=\
            torch.cat((discountParameterTensorofAllTimeSteps,discountParameterTensorInOneTimeStep.unsqueeze(0)),dim=0)
    return relationTensorOfAllTimeSteps,discountParameterTensorofAllTimeSteps
    
            
            


In [19]:
tensor1=torch.rand((6))
tensor2=torch.rand((6))
tensor3=torch.rand((6))
tensor4=torch.stack((tensor1,tensor2,tensor3),dim=0)
logging.debug(tensor1)
logging.debug(tensor2)
logging.debug(tensor3)
logging.debug(tensor4)
tensor5=torch.cat((tensor1.unsqueeze(0),tensor4),dim=0)
logging.debug(tensor5)

2020-08-02 22:25:03,284 -5 - <module> - DEBUG - tensor([0.6510, 0.1431, 0.1494, 0.7126, 0.2595, 0.9902])
2020-08-02 22:25:03,304 -6 - <module> - DEBUG - tensor([0.9832, 0.8018, 0.9962, 0.6808, 0.8797, 0.0326])
2020-08-02 22:25:03,305 -7 - <module> - DEBUG - tensor([0.2613, 0.6710, 0.9968, 0.1821, 0.1820, 0.4792])
2020-08-02 22:25:03,306 -8 - <module> - DEBUG - tensor([[0.6510, 0.1431, 0.1494, 0.7126, 0.2595, 0.9902],
        [0.9832, 0.8018, 0.9962, 0.6808, 0.8797, 0.0326],
        [0.2613, 0.6710, 0.9968, 0.1821, 0.1820, 0.4792]])
2020-08-02 22:25:03,307 -10 - <module> - DEBUG - tensor([[0.6510, 0.1431, 0.1494, 0.7126, 0.2595, 0.9902],
        [0.6510, 0.1431, 0.1494, 0.7126, 0.2595, 0.9902],
        [0.9832, 0.8018, 0.9962, 0.6808, 0.8797, 0.0326],
        [0.2613, 0.6710, 0.9968, 0.1821, 0.1820, 0.4792]])


In [20]:
def differenceBetweenTwoFrame(frameSeries):
    """
    Given a series of frame, return the difference series of those frame. Since this function 
    compute diffrences by the gap between two adjacent frames, the quantity of frame in difference 
    series is one less than the input frame series
    Args:
        frameSeries: input frames, which dimension is (vehicleQuantity, vehicleProperties)
    Returns:
        the difference series, which first dimension (quantity dimension) is one less than input frame series.
    """
    frameSeriesWithoutTheFirstFrame=frameSeries[1:]
    frameSeriesWithoutTheLastFrame=frameSeries[0:-1]
    logging.debug(str(frameSeriesWithoutTheFirstFrame))
    logging.debug(str(frameSeriesWithoutTheLastFrame))
    logging.debug(str(frameSeries))
    differenceSeries=frameSeriesWithoutTheFirstFrame-frameSeriesWithoutTheLastFrame
    return differenceSeries

In [21]:
def differenceBetweenTwoFrameForBatch(frameSeries):
    """
    Given batches of series of frame, return the difference series of all those frames. Since this function 
    compute diffrences by the gap between two adjacent frames, the quantity of frame in difference 
    series is one less than the input frame series
    Args:
        frameSeries: input frames, which dimension is (batch,vehicleQuantity, vehicleProperties)
    Returns:
        the difference series, which first dimension (quantity dimension) is one less than input frame series.
    """
    
    frameSeriesWithoutTheFirstFrame=frameSeries[:,1:]
    frameSeriesWithoutTheLastFrame=frameSeries[:,0:-1]
    logging.debug(str(frameSeriesWithoutTheFirstFrame))
    logging.debug(str(frameSeriesWithoutTheLastFrame))
    logging.debug(str(frameSeries))
    differenceSeries=frameSeriesWithoutTheFirstFrame-frameSeriesWithoutTheLastFrame
    return differenceSeries

In [22]:
inputTensor=torch.rand((5,5))
logging.debug(inputTensor)
differenceBetweenTwoFrame(inputTensor)
logging.debug(fromAllToStr('shapesize',inputTensor.shape.__len__()))

2020-08-02 22:25:03,325 -2 - <module> - DEBUG - tensor([[0.1419, 0.9246, 0.9421, 0.6925, 0.6367],
        [0.5708, 0.0687, 0.6147, 0.7858, 0.4763],
        [0.9298, 0.6648, 0.3424, 0.3496, 0.4910],
        [0.0116, 0.6750, 0.0485, 0.8035, 0.3734],
        [0.9485, 0.4068, 0.2723, 0.8176, 0.0725]])
2020-08-02 22:25:03,328 -13 - differenceBetweenTwoFrame - DEBUG - tensor([[0.5708, 0.0687, 0.6147, 0.7858, 0.4763],
        [0.9298, 0.6648, 0.3424, 0.3496, 0.4910],
        [0.0116, 0.6750, 0.0485, 0.8035, 0.3734],
        [0.9485, 0.4068, 0.2723, 0.8176, 0.0725]])
2020-08-02 22:25:03,328 -14 - differenceBetweenTwoFrame - DEBUG - tensor([[0.1419, 0.9246, 0.9421, 0.6925, 0.6367],
        [0.5708, 0.0687, 0.6147, 0.7858, 0.4763],
        [0.9298, 0.6648, 0.3424, 0.3496, 0.4910],
        [0.0116, 0.6750, 0.0485, 0.8035, 0.3734]])
2020-08-02 22:25:03,329 -15 - differenceBetweenTwoFrame - DEBUG - tensor([[0.1419, 0.9246, 0.9421, 0.6925, 0.6367],
        [0.5708, 0.0687, 0.6147, 0.7858, 0.4763],
 

In [23]:
# testExpand=torch.tensor(((1,2,3),(3,4,5))).expand(2,6)
# print(testExpand)
# theList=[1,2,3,5,7]
# theList.remove(7)
# d = {'lilee':25, 'wangyan':21, 'liqun':32, 'age':19}
# print(d)
# d=sorted(d.items(), key=lambda item:item[1])
# print(d.__len__())
# print(theList)
# testTensor=torch.rand(10,10)
# print(testTensor)
# print(testTensor[[1,4,3,4]])

In [24]:
#save visulized images
# for key in list(valueDict.keys())[1:10000]:
#     image=visualizeData(valueDict[key])
#     cv2.imwrite('visualizeFolder/image'+str(key)+'.png',image)

In [25]:
class tensorsDataset(Dataset):
    def __init__(self, trajectoryFileList, numberOfTensorsEachBatch=100,lableTensorEachBatch=2):
        if(numberOfTensorsEachBatch<5):
            raise Exception("THE NUMBER OF TENSORS IN EACH BATCH IS TOO SMALL")
        self.valueDictList=[]
        self.sizeList=[]
        self.numberOfTensorsEachBatch=numberOfTensorsEachBatch
        self.lableTensorEachBatch=lableTensorEachBatch
        self.allTensorsEachBatch=numberOfTensorsEachBatch+lableTensorEachBatch
        self.keysList=[]
        for eachFile in trajectoryFileList:
            valueDict=fromDirGenerateDict(eachFile)
            self.valueDictList.append(copy.deepcopy(valueDict))
            self.sizeList.append(valueDict.keys().__len__()-self.allTensorsEachBatch)
            sortedKeys=list(valueDict.keys())
            sortedKeys.sort()
            self.keysList.append(copy.deepcopy(sortedKeys))

    def __len__(self):
        allLen=0
        for length in self.sizeList:
            allLen=allLen+length
        return allLen
    
    def fromIdxMapToList(self,idx):
        """
        since there are several lists,we have to know which list shoud we use and the ture index in the list
        Return:
            the trueIndex, listIndex
        """
        countSection=0
        for i in range(0,self.sizeList.__len__()):
            countSection+=self.sizeList[i]
            if(idx<countSection):
                return idx-countSection+self.sizeList[i],i
            
    
    def __getitem__(self,idx):
        #initialize trueIndex, listIndex
        #
        #this dict record the relation between matrix index and vehicle ID in the last frame
        matrixIndexAndVehicleIDRecordDict={}
        #initialize dict above
        for i in range(0,maxMatrixIndex):
            matrixIndexAndVehicleIDRecordDict[i]={'Vehicle_ID':-1,'refresh':-1}
        matrixIndexAndVehicleIDRecordDict['time']=-1
        trueIndex,listIndex=self.fromIdxMapToList(idx)
        itemDict={'positionTensorList':[],'speedTensorList':[],'accTensorList':[],'angleTensorList':[],'time':[]}
        valueDict=self.valueDictList[listIndex] #valueDict is the Dict of many frames
        dictKeys=self.keysList[listIndex]
        #generate tensors of first frame
        positionTensor,speedTensor,accTensor,angleTensor,newVehicleList\
        =readFirstFrame(matrixIndexAndVehicleIDRecordDict,valueDict[dictKeys[trueIndex]])
        angleSinTensor=torch.sin(angleTensor)
        angleCosTensor=torch.cos(angleTensor)
        combinedTensor=torch.cat((positionTensor,speedTensor.mul(angleSinTensor),\
                                 speedTensor.mul(angleCosTensor),accTensor.mul(angleSinTensor),\
                                 accTensor.mul(angleCosTensor)),0)
        allCombineTensorTrain=combinedTensor.unsqueeze(0)
        allCombineTensorValid=0
#         itemDict['positionTensorList'].append(positionTensor)
#         itemDict['speedTensorList'].append(speedTensor)
#         itemDict['accTensorList'].append(accTensor)
#         itemDict['angleTensorList'].append(angleTensor)
#         time=getValueByLable(['Global_Time'],valueDict[dictKeys[trueIndex]][0]) #valueDict[sortedDitKey[theIndexOfSortedDictKey]][0]
#         itemDict['time'].append(time)
        
        for i in range(trueIndex+1,trueIndex+self.allTensorsEachBatch):
            #generate tensor from general frame
            positionTensor,speedTensor,accTensor,angleTensor,newVehicleList,vanishedVehicleList\
            =readGeneralFrame(matrixIndexAndVehicleIDRecordDict,valueDict[dictKeys[i]],positionTensor)
            angleSinTensor=torch.sin(angleTensor)
            angleCosTensor=torch.cos(angleTensor)
            combinedTensor=torch.cat((positionTensor,speedTensor.mul(angleSinTensor),\
                                     speedTensor.mul(angleCosTensor),accTensor.mul(angleSinTensor),\
                                     accTensor.mul(angleCosTensor)),0)
            if i<self.numberOfTensorsEachBatch+trueIndex:
                allCombineTensorTrain=torch.cat((allCombineTensorTrain,combinedTensor.unsqueeze(0)),0)
            elif i==self.numberOfTensorsEachBatch+trueIndex:
                allCombineTensorValid=combinedTensor.unsqueeze(0)
            else:
                allCombineTensorValid=torch.cat((allCombineTensorValid,combinedTensor.unsqueeze(0)),0)
#             itemDict['positionTensorList'].append(positionTensor)
#             itemDict['speedTensorList'].append(speedTensor)
#             itemDict['accTensorList'].append(accTensor)
#             itemDict['angleTensorList'].append(angleTensor)
#             time=getValueByLable(['Global_Time'],valueDict[dictKeys[i]][0]) #valueDict[sortedDitKey[theIndexOfSortedDictKey]]
#             itemDict['time'].append(time)
        return allCombineTensorTrain,allCombineTensorValid
# trajectoryFileList=['/home/wangyuchen/trajectory_dataset/US101/0750am-0805am/trajectories-0750am-0805am.txt',\
#                    '/home/wangyuchen/trajectory_dataset/US101/0805am-0820am/trajectories-0805am-0820am.txt',\
#                    '/home/wangyuchen/trajectory_dataset/US101/0820am-0835am/trajectories-0820am-0835am.txt']
trajectoryFileList=['/home/wangyuchen/trajectory_dataset/US101/0750am-0805am/trajectories-0750am-0805am.txt']


In [26]:
import math
class tensorsDatasetV2(Dataset):
    """
    dataset for relation model
    """
    def __init__(self, trajectoryFileList, numberOfTensorsEachBatch=1,lableTensorEachBatch=1):
        if(numberOfTensorsEachBatch!=1 or lableTensorEachBatch!=1):
            raise Exception("BOTH TRAIN AND VALID TENSOR NUMBERS SHOULD BE ONE!")
        self.valueDictList=[]
        self.sizeList=[]
        self.numberOfTensorsEachBatch=numberOfTensorsEachBatch
        self.lableTensorEachBatch=lableTensorEachBatch
        self.allTensorsEachBatch=numberOfTensorsEachBatch+lableTensorEachBatch
        self.keysList=[]
        if doNormalization:
            self.positionXMax=0
            self.positionXMin=999999
            self.positionYMax=0
            self.positionYMin=99999
            self.speedMax=-100
            self.speedMin=999999
            self.accMax=-100
            self.accMin=9999
        for eachFile in trajectoryFileList:
            valueDict=fromDirGenerateDict(eachFile)
            if doNormalization:
                #get the max and min value for normalization
                maxAndMinDict=getMaxAndMinValueFromValueDict(valueDict,['Local_X','Local_Y','v_Vel','v_Acc'])
                #position X
                if self.positionXMax<maxAndMinDict['Local_X']['max']:
                    self.positionXMax=maxAndMinDict['Local_X']['max']
                if self.positionXMin>maxAndMinDict['Local_X']['min']:
                    self.positionXMin=maxAndMinDict['Local_X']['min']
                #position Y
                if self.positionYMax<maxAndMinDict['Local_Y']['max']:
                    self.positionYMax=maxAndMinDict['Local_Y']['max']
                if self.positionYMin>maxAndMinDict['Local_Y']['min']:
                    self.positionYMin=maxAndMinDict['Local_Y']['min']
                #speed
                if self.speedMax<maxAndMinDict['v_Vel']['max']:
                    self.speedMax=maxAndMinDict['v_Vel']['max']
                if self.speedMin>maxAndMinDict['v_Vel']['min']:
                    self.speedMin=maxAndMinDict['v_Vel']['min']
                #acc
                if self.accMax<maxAndMinDict['v_Acc']['max']:
                    self.accMax=maxAndMinDict['v_Acc']['max']
                if self.accMin>maxAndMinDict['v_Acc']['min']:
                    self.accMin=maxAndMinDict['v_Acc']['min']
            self.valueDictList.append(copy.deepcopy(valueDict))
            self.sizeList.append(valueDict.keys().__len__()-self.allTensorsEachBatch)
            sortedKeys=list(valueDict.keys())
            sortedKeys.sort()
            self.keysList.append(copy.deepcopy(sortedKeys))

    def getNormalizationDict(self):
        '''
        Returns:
            a dict:{'positionXMax':self.positionXMax,'positonYMax':self.self.positionYMax,\
               'positionXMin':self.positionXMin,'positionYMin':self.positionYMin,\
               'speedMax':self.speedMax,'speedMin':speedMin,\
               'accMax':self.accMax,'accMax':self.accMin}
        '''
        if not doNormalization:
            raise Exception('NORMALIZATION IS NOT APPLIED')
        return {'positionXMax':self.positionXMax,'positionYMax':self.positionYMax,\
               'positionXMin':self.positionXMin,'positionYMin':self.positionYMin,\
               'speedMax':self.speedMax,'speedMin':self.speedMin,\
               'accMax':self.accMax,'accMin':self.accMin}
    
    def __len__(self):
        allLen=0
        for length in self.sizeList:
            allLen=allLen+length
        return allLen
    
    def fromIdxMapToList(self,idx):
        """
        since there are several lists,we have to know which list shoud we use and the ture index in the list
        Return:
            the trueIndex, listIndex
        """
        countSection=0
        for i in range(0,self.sizeList.__len__()):
            countSection+=self.sizeList[i]
            if(idx<countSection):
                return idx-countSection+self.sizeList[i],i
            
    
            
    
    def __getitem__(self,idx):
        #initialize trueIndex, listIndex
        #
        #this dict record the relation between matrix index and vehicle ID in the last frame
        matrixIndexAndVehicleIDRecordDict={}
        #initialize dict above
        for i in range(0,maxMatrixIndex):
            matrixIndexAndVehicleIDRecordDict[i]={'Vehicle_ID':-1,'refresh':-1}
        matrixIndexAndVehicleIDRecordDict['time']=-1
        trueIndex,listIndex=self.fromIdxMapToList(idx)
        itemDict={'positionTensorList':[],'speedTensorList':[],'accTensorList':[],'angleTensorList':[],'time':[]}
        valueDict=self.valueDictList[listIndex] #valueDict is the Dict of many frames
        dictKeys=self.keysList[listIndex]
        #generate tensors of first frame
        positionTensor,speedTensor,accTensor,angleTensor,newVehicleList\
        =readFirstFrame(matrixIndexAndVehicleIDRecordDict,valueDict[dictKeys[trueIndex]])
        angleSinTensor=torch.sin(angleTensor)
        angleCosTensor=torch.cos(angleTensor)
        #first frame normalization
        if doNormalization:
#             print('before nomalization')
#             print(positionTensor.shape,speedTensor.shape,accTensor.shape)
            positionTensor=torch.cat((torch.div(torch.sub(positionTensor[0,:],self.positionXMin),self.positionXMax-self.positionXMin).unsqueeze(0),\
                                     torch.div(torch.sub(positionTensor[1,:],self.positionYMin,),self.positionYMax-self.positionYMin).unsqueeze(0)),0)
            speedTensor=torch.div(torch.sub(speedTensor,self.speedMin),self.speedMax-self.speedMin)
            accTensor=torch.div(torch.sub(accTensor,self.accMin),self.accMax-self.accMin)
#             print('after normalization')
#             print(positionTensor.shape,speedTensor.shape,accTensor.shape)
            combinedTensor=torch.cat((positionTensor,speedTensor.mul(angleSinTensor),\
                                     speedTensor.mul(angleCosTensor),accTensor.mul(angleSinTensor),\
                                     accTensor.mul(angleCosTensor)),0)
        else:
            combinedTensor=torch.cat((positionTensor,speedTensor.mul(angleSinTensor),\
                                     speedTensor.mul(angleCosTensor),accTensor.mul(angleSinTensor),\
                                     accTensor.mul(angleCosTensor)),0)
        #generate relation tensor for all vehicle pairs
        print('in getitem, combinedTensor shape: ',combinedTensor.shape)
        relationTensorLeft=combinedTensor[:,0].expand(combinedTensor.shape[1]-1,combinedTensor.shape[0])
        relationTensorRight=torch.transpose(torch.cat((combinedTensor[:,:0],combinedTensor[:,1:]),1),0,1)
        print('in getitem, relation tensorleft shape:',relationTensorLeft.shape)
        print('in getitem, relationtensorright shape',relationTensorRight.shape)
#         print(relationTensorRight.shape,relationTensorRight.shape)
        for i in range(1,combinedTensor.shape[1]):
            relationTensorLeft=torch.cat((relationTensorLeft,\
                                          combinedTensor[:,i].expand(combinedTensor.shape[1]-1,combinedTensor.shape[0])),0)
            relationTensorRight=torch.cat((relationTensorRight,\
                                           torch.transpose(torch.cat((combinedTensor[:,:i],combinedTensor[:,i+1:]),1),0,1)),0)
#         print(relationTensorLeft.shape,relationTensorRight.shape)
        combinedRelationTensor=torch.cat((relationTensorLeft,relationTensorRight),1)  
        firstCombinedRelationTensor=combinedRelationTensor
        
        
        allCombineTensorTrain=combinedTensor.unsqueeze(0)
        allCombineTensorValid=0
#         itemDict['positionTensorList'].append(positionTensor)
#         itemDict['speedTensorList'].append(speedTensor)
#         itemDict['accTensorList'].append(accTensor)
#         itemDict['angleTensorList'].append(angleTensor)
#         time=getValueByLable(['Global_Time'],valueDict[dictKeys[trueIndex]][0]) #valueDict[sortedDitKey[theIndexOfSortedDictKey]][0]
#         itemDict['time'].append(time)
        
        for i in range(trueIndex+1,trueIndex+self.allTensorsEachBatch):
            #generate tensor from general frame
            positionTensor,speedTensor,accTensor,angleTensor,newVehicleList,vanishedVehicleList\
            =readGeneralFrame(matrixIndexAndVehicleIDRecordDict,valueDict[dictKeys[i]],positionTensor)
            angleSinTensor=torch.sin(angleTensor)
            angleCosTensor=torch.cos(angleTensor)
            if doNormalization:
                positionTensor=torch.cat((torch.div(torch.sub(positionTensor[0,:],self.positionXMin),self.positionXMax-self.positionXMin).unsqueeze(0),\
                                         torch.div(torch.sub(positionTensor[1,:],self.positionYMin,),self.positionYMax-self.positionYMin).unsqueeze(0)),0)
                speedTensor=torch.div(torch.sub(speedTensor,self.speedMin),self.speedMax-self.speedMin)
                accTensor=torch.div(torch.sub(accTensor,self.accMin),self.accMax-self.accMin)
                combinedTensor=torch.cat((positionTensor,speedTensor.mul(angleSinTensor),\
                                         speedTensor.mul(angleCosTensor),accTensor.mul(angleSinTensor),\
                                         accTensor.mul(angleCosTensor)),0)
            else:
                combinedTensor=torch.cat((positionTensor,speedTensor.mul(angleSinTensor),\
                                     speedTensor.mul(angleCosTensor),accTensor.mul(angleSinTensor),\
                                         accTensor.mul(angleCosTensor)),0)
            #generate relation tensor for all vehicle pairs
            relationTensorLeft=combinedTensor[:,0].expand(combinedTensor.shape[1]-1,combinedTensor.shape[0])
            relationTensorRight=torch.transpose(torch.cat((combinedTensor[:,:0],combinedTensor[:,1:]),1),0,1)
#             print(relationTensorRight.shape,relationTensorRight.shape)
            for j in range(1,combinedTensor.shape[1]):
                relationTensorLeft=torch.cat((relationTensorLeft,\
                                              combinedTensor[:,j].expand(combinedTensor.shape[1]-1,combinedTensor.shape[0])),0)
                relationTensorRight=torch.cat((relationTensorRight,\
                                               torch.transpose(torch.cat((combinedTensor[:,:j],combinedTensor[:,j+1:]),1),0,1)),0)
#             print(relationTensorLeft.shape,relationTensorRight.shape)
            combinedRelationTensor=torch.cat((relationTensorLeft,relationTensorRight),1)  
            secondRelationTensor=combinedRelationTensor
            #since we only need two tensors, which is input and output tensor respectively, we could return
            #the two tensors in the first loop
            #(ok I admit that the true reason is that I am lazy)
            return firstCombinedRelationTensor,secondRelationTensor
#             if i<self.numberOfTensorsEachBatch+trueIndex:
#                 allCombineTensorTrain=torch.cat((allCombineTensorTrain,combinedTensor.unsqueeze(0)),0)
#             elif i==self.numberOfTensorsEachBatch+trueIndex:
#                 allCombineTensorValid=combinedTensor.unsqueeze(0)
#             else:
#                 allCombineTensorValid=torch.cat((allCombineTensorValid,combinedTensor.unsqueeze(0)),0)
#             itemDict['positionTensorList'].append(positionTensor)
#             itemDict['speedTensorList'].append(speedTensor)
#             itemDict['accTensorList'].append(accTensor)
#             itemDict['angleTensorList'].append(angleTensor)
#             time=getValueByLable(['Global_Time'],valueDict[dictKeys[i]][0]) #valueDict[sortedDitKey[theIndexOfSortedDictKey]]
#             itemDict['time'].append(time)
        return allCombineTensorTrain,allCombineTensorValid
# trajectoryFileList=['/home/wangyuchen/trajectory_dataset/US101/0750am-0805am/trajectories-0750am-0805am.txt',\
#                    '/home/wangyuchen/trajectory_dataset/US101/0805am-0820am/trajectories-0805am-0820am.txt',\
#                    '/home/wangyuchen/trajectory_dataset/US101/0820am-0835am/trajectories-0820am-0835am.txt']
trajectoryFileList=['/home/wangyuchen/trajectory_dataset/US101/0750am-0805am/trajectories-0750am-0805am.txt']
#run on 2080 in g814
if runOnG814:
    trajectoryFileList=['/home/wangyuchen/US101/0750am-0805am/trajectories-0750am-0805am.txt']


In [27]:
def fromObjectsToRelationPairsBatchAndTimestepVersion(batchAndTimestepCombinedTensor):
    '''
    This function based on the other function termed as 'fromObjectsToRelationPairs'. Instead of process a 
    single frame, this function takes batch and timestep(the other dimension) into consideration.
    note: the dimension of combinedTensor is supposed to be (batchs, timesteps, properties, vehicles)
    Args:
        The input tensor should already be transposed if it is generated from the network's output
    Returns:
        Relation pairs
    '''
    #generate relation tensor for all vehicle pairs
    batchSize=batchAndTimestepCombinedTensor.shape[0]
    timesteps=batchAndTimestepCombinedTensor.shape[1]
    for batch in range(batchSize):
        for timestep in range(timesteps):
            combinedTensor=batchAndTimestepCombinedTensor[batch,timestep,:,:]
            relationTensorLeft=combinedTensor[:,0].expand(combinedTensor.shape[1]-1,combinedTensor.shape[0])
            relationTensorRight=torch.transpose(torch.cat((combinedTensor[:,:0],combinedTensor[:,1:]),1),0,1)
        #         print(relationTensorRight.shape,relationTensorRight.shape)
            for i in range(1,combinedTensor.shape[1]):
                relationTensorLeft=torch.cat((relationTensorLeft,\
                                              combinedTensor[:,i].expand(combinedTensor.shape[1]-1,combinedTensor.shape[0])),0)
                relationTensorRight=torch.cat((relationTensorRight,\
                                               torch.transpose(torch.cat((combinedTensor[:,:i],combinedTensor[:,i+1:]),1),0,1)),0)
        #         print(relationTensorLeft.shape,relationTensorRight.shape)
            combinedRelationTensor=torch.cat((relationTensorLeft,relationTensorRight),1) 
            if timestep==0:
                combineRelationTensorsTimeStep=combinedRelationTensor.unsqueeze(0)
            else:
                combineRelationTensorsTimeStep=torch.cat((combineRelationTensorsTimeStep,\
                                                          combinedRelationTensor.unsqueeze(0)),0)
        if batch==0:
            combinedRelationTensorsTimeStepAndBatch=combineRelationTensorsTimeStep.unsqueeze(0)
        else:
            combinedRelationTensorsTimeStepAndBatch=torch.cat((combinedRelationTensorsTimeStepAndBatch,\
                                                              combineRelationTensorsTimeStep.unsqueeze(0)),0)
    return combinedRelationTensorsTimeStepAndBatch

In [28]:
#test the combined tensor and relation tensor i tensorsDataV2
import math
class tensorsDatasetV2Test(Dataset):
    """
    dataset for relation model
    """
    def __init__(self, trajectoryFileList, numberOfTensorsEachBatch=1,lableTensorEachBatch=1):
        if(numberOfTensorsEachBatch!=1 or lableTensorEachBatch!=1):
            raise Exception("BOTH TRAIN AND VALID TENSOR NUMBERS SHOULD BE ONE!")
        self.valueDictList=[]
        self.sizeList=[]
        self.numberOfTensorsEachBatch=numberOfTensorsEachBatch
        self.lableTensorEachBatch=lableTensorEachBatch
        self.allTensorsEachBatch=numberOfTensorsEachBatch+lableTensorEachBatch
        self.keysList=[]
        if doNormalization:
            self.positionXMax=0
            self.positionXMin=999999
            self.positionYMax=0
            self.positionYMin=99999
            self.speedMax=-100
            self.speedMin=999999
            self.accMax=-100
            self.accMin=9999
        for eachFile in trajectoryFileList:
            valueDict=fromDirGenerateDict(eachFile)
            if doNormalization:
                #get the max and min value for normalization
                maxAndMinDict=getMaxAndMinValueFromValueDict(valueDict,['Local_X','Local_Y','v_Vel','v_Acc'])
                #position X
                if self.positionXMax<maxAndMinDict['Local_X']['max']:
                    self.positionXMax=maxAndMinDict['Local_X']['max']
                if self.positionXMin>maxAndMinDict['Local_X']['min']:
                    self.positionXMin=maxAndMinDict['Local_X']['min']
                #position Y
                if self.positionYMax<maxAndMinDict['Local_Y']['max']:
                    self.positionYMax=maxAndMinDict['Local_Y']['max']
                if self.positionYMin>maxAndMinDict['Local_Y']['min']:
                    self.positionYMin=maxAndMinDict['Local_Y']['min']
                #speed
                if self.speedMax<maxAndMinDict['v_Vel']['max']:
                    self.speedMax=maxAndMinDict['v_Vel']['max']
                if self.speedMin>maxAndMinDict['v_Vel']['min']:
                    self.speedMin=maxAndMinDict['v_Vel']['min']
                #acc
                if self.accMax<maxAndMinDict['v_Acc']['max']:
                    self.accMax=maxAndMinDict['v_Acc']['max']
                if self.accMin>maxAndMinDict['v_Acc']['min']:
                    self.accMin=maxAndMinDict['v_Acc']['min']
            self.valueDictList.append(copy.deepcopy(valueDict))
            self.sizeList.append(valueDict.keys().__len__()-self.allTensorsEachBatch)
            sortedKeys=list(valueDict.keys())
            sortedKeys.sort()
            self.keysList.append(copy.deepcopy(sortedKeys))

    def getNormalizationDict(self):
        '''
        Returns:
            a dict:{'positionXMax':self.positionXMax,'positonYMax':self.self.positionYMax,\
               'positionXMin':self.positionXMin,'positionYMin':self.positionYMin,\
               'speedMax':self.speedMax,'speedMin':speedMin,\
               'accMax':self.accMax,'accMax':self.accMin}
        '''
        if not doNormalization:
            raise Exception('NORMALIZATION IS NOT APPLIED')
        return {'positionXMax':self.positionXMax,'positionYMax':self.positionYMax,\
               'positionXMin':self.positionXMin,'positionYMin':self.positionYMin,\
               'speedMax':self.speedMax,'speedMin':self.speedMin,\
               'accMax':self.accMax,'accMin':self.accMin}
    
    def __len__(self):
        allLen=0
        for length in self.sizeList:
            allLen=allLen+length
        return allLen
    
    def fromIdxMapToList(self,idx):
        """
        since there are several lists,we have to know which list shoud we use and the ture index in the list
        Return:
            the trueIndex, listIndex
        """
        countSection=0
        for i in range(0,self.sizeList.__len__()):
            countSection+=self.sizeList[i]
            if(idx<countSection):
                return idx-countSection+self.sizeList[i],i
            
    
            
    
    def __getitem__(self,idx):
        #initialize trueIndex, listIndex
        #
        #this dict record the relation between matrix index and vehicle ID in the last frame
        matrixIndexAndVehicleIDRecordDict={}
        #initialize dict above
        for i in range(0,maxMatrixIndex):
            matrixIndexAndVehicleIDRecordDict[i]={'Vehicle_ID':-1,'refresh':-1}
        matrixIndexAndVehicleIDRecordDict['time']=-1
        trueIndex,listIndex=self.fromIdxMapToList(idx)
        itemDict={'positionTensorList':[],'speedTensorList':[],'accTensorList':[],'angleTensorList':[],'time':[]}
        valueDict=self.valueDictList[listIndex] #valueDict is the Dict of many frames
        dictKeys=self.keysList[listIndex]
        #generate tensors of first frame
        positionTensor,speedTensor,accTensor,angleTensor,newVehicleList\
        =readFirstFrame(matrixIndexAndVehicleIDRecordDict,valueDict[dictKeys[trueIndex]])
        angleSinTensor=torch.sin(angleTensor)
        angleCosTensor=torch.cos(angleTensor)
        fileName='./'+'tensorFromGetitem'+'/'+str(10000000+idx)+'.png'
        image=visualizeTensorData(positionTensor[0,:],positionTensor[1,:])
        cv2.imwrite(fileName,image)
        #first frame normalization
        if doNormalization:
#             print('before nomalization')
#             print(positionTensor.shape,speedTensor.shape,accTensor.shape)
            positionTensor=torch.cat((torch.div(torch.sub(positionTensor[0,:],self.positionXMin),self.positionXMax-self.positionXMin).unsqueeze(0),\
                                     torch.div(torch.sub(positionTensor[1,:],self.positionYMin,),self.positionYMax-self.positionYMin).unsqueeze(0)),0)
            speedTensor=torch.div(torch.sub(speedTensor,self.speedMin),self.speedMax-self.speedMin)
            accTensor=torch.div(torch.sub(accTensor,self.accMin),self.accMax-self.accMin)
#             print('after normalization')
#             print(positionTensor.shape,speedTensor.shape,accTensor.shape)
            combinedTensor=torch.cat((positionTensor,speedTensor.mul(angleSinTensor),\
                                     speedTensor.mul(angleCosTensor),accTensor.mul(angleSinTensor),\
                                     accTensor.mul(angleCosTensor)),0)
        else:
            combinedTensor=torch.cat((positionTensor,speedTensor.mul(angleSinTensor),\
                                     speedTensor.mul(angleCosTensor),accTensor.mul(angleSinTensor),\
                                     accTensor.mul(angleCosTensor)),0)
        #generate relation tensor for all vehicle pairs
        print('in getitem, combinedTensor shape: ',combinedTensor.shape)
        fileName='./'+'tensorFromGetitemAfterNormalization'+'/'+str(10000000+idx)+'.png'
        image=visualizeTensorData(positionTensor[0,:],positionTensor[1,:],normalizationDict=self.getNormalizationDict())
        cv2.imwrite(fileName,image)
        relationTensorLeft=combinedTensor[:,0].expand(combinedTensor.shape[1]-1,combinedTensor.shape[0])
        relationTensorRight=torch.transpose(torch.cat((combinedTensor[:,:0],combinedTensor[:,1:]),1),0,1)
        print('in getitem, relation tensorleft shape:',relationTensorLeft.shape)
        print('in getitem, relationtensorright shape',relationTensorRight.shape)
#         print(relationTensorRight.shape,relationTensorRight.shape)
        for i in range(1,combinedTensor.shape[1]):
            relationTensorLeft=torch.cat((relationTensorLeft,\
                                          combinedTensor[:,i].expand(combinedTensor.shape[1]-1,combinedTensor.shape[0])),0)
            relationTensorRight=torch.cat((relationTensorRight,\
                                           torch.transpose(torch.cat((combinedTensor[:,:i],combinedTensor[:,i+1:]),1),0,1)),0)
#         print(relationTensorLeft.shape,relationTensorRight.shape)
        combinedRelationTensor=torch.cat((relationTensorLeft,relationTensorRight),1)  
        firstCombinedRelationTensor=combinedRelationTensor
        firstCombinedTensor=combinedTensor
        
        
        allCombineTensorTrain=combinedTensor.unsqueeze(0)
        allCombineTensorValid=0
#         itemDict['positionTensorList'].append(positionTensor)
#         itemDict['speedTensorList'].append(speedTensor)
#         itemDict['accTensorList'].append(accTensor)
#         itemDict['angleTensorList'].append(angleTensor)
#         time=getValueByLable(['Global_Time'],valueDict[dictKeys[trueIndex]][0]) #valueDict[sortedDitKey[theIndexOfSortedDictKey]][0]
#         itemDict['time'].append(time)
        
        for i in range(trueIndex+1,trueIndex+self.allTensorsEachBatch):
            #generate tensor from general frame
            positionTensor,speedTensor,accTensor,angleTensor,newVehicleList,vanishedVehicleList\
            =readGeneralFrame(matrixIndexAndVehicleIDRecordDict,valueDict[dictKeys[i]],positionTensor)
            angleSinTensor=torch.sin(angleTensor)
            angleCosTensor=torch.cos(angleTensor)
            if doNormalization:
                positionTensor=torch.cat((torch.div(torch.sub(positionTensor[0,:],self.positionXMin),self.positionXMax-self.positionXMin).unsqueeze(0),\
                                         torch.div(torch.sub(positionTensor[1,:],self.positionYMin,),self.positionYMax-self.positionYMin).unsqueeze(0)),0)
                speedTensor=torch.div(torch.sub(speedTensor,self.speedMin),self.speedMax-self.speedMin)
                accTensor=torch.div(torch.sub(accTensor,self.accMin),self.accMax-self.accMin)
                combinedTensor=torch.cat((positionTensor,speedTensor.mul(angleSinTensor),\
                                         speedTensor.mul(angleCosTensor),accTensor.mul(angleSinTensor),\
                                         accTensor.mul(angleCosTensor)),0)
            else:
                combinedTensor=torch.cat((positionTensor,speedTensor.mul(angleSinTensor),\
                                     speedTensor.mul(angleCosTensor),accTensor.mul(angleSinTensor),\
                                         accTensor.mul(angleCosTensor)),0)
            #generate relation tensor for all vehicle pairs
            relationTensorLeft=combinedTensor[:,0].expand(combinedTensor.shape[1]-1,combinedTensor.shape[0])
            relationTensorRight=torch.transpose(torch.cat((combinedTensor[:,:0],combinedTensor[:,1:]),1),0,1)
#             print(relationTensorRight.shape,relationTensorRight.shape)
            for j in range(1,combinedTensor.shape[1]):
                relationTensorLeft=torch.cat((relationTensorLeft,\
                                              combinedTensor[:,j].expand(combinedTensor.shape[1]-1,combinedTensor.shape[0])),0)
                relationTensorRight=torch.cat((relationTensorRight,\
                                               torch.transpose(torch.cat((combinedTensor[:,:j],combinedTensor[:,j+1:]),1),0,1)),0)
#             print(relationTensorLeft.shape,relationTensorRight.shape)
            combinedRelationTensor=torch.cat((relationTensorLeft,relationTensorRight),1)  
            secondRelationTensor=combinedRelationTensor
            secondCombinedTensor=combinedTensor
            #since we only need two tensors, which is input and output tensor respectively, we could return
            #the two tensors in the first loop
            #(ok I admit that the true reason is that I am lazy)
            return firstCombinedTensor,secondCombinedTensor
            return firstCombinedRelationTensor,secondRelationTensor
#             if i<self.numberOfTensorsEachBatch+trueIndex:
#                 allCombineTensorTrain=torch.cat((allCombineTensorTrain,combinedTensor.unsqueeze(0)),0)
#             elif i==self.numberOfTensorsEachBatch+trueIndex:
#                 allCombineTensorValid=combinedTensor.unsqueeze(0)
#             else:
#                 allCombineTensorValid=torch.cat((allCombineTensorValid,combinedTensor.unsqueeze(0)),0)
#             itemDict['positionTensorList'].append(positionTensor)
#             itemDict['speedTensorList'].append(speedTensor)
#             itemDict['accTensorList'].append(accTensor)
#             itemDict['angleTensorList'].append(angleTensor)
#             time=getValueByLable(['Global_Time'],valueDict[dictKeys[i]][0]) #valueDict[sortedDitKey[theIndexOfSortedDictKey]]
#             itemDict['time'].append(time)
        return allCombineTensorTrain,allCombineTensorValid
# trajectoryFileList=['/home/wangyuchen/trajectory_dataset/US101/0750am-0805am/trajectories-0750am-0805am.txt',\
#                    '/home/wangyuchen/trajectory_dataset/US101/0805am-0820am/trajectories-0805am-0820am.txt',\
#                    '/home/wangyuchen/trajectory_dataset/US101/0820am-0835am/trajectories-0820am-0835am.txt']
trajectoryFileList=['/home/wangyuchen/trajectory_dataset/US101/0750am-0805am/trajectories-0750am-0805am.txt']
#run on 2080 in g814
if runOnG814:
    trajectoryFileList=['/home/wangyuchen/US101/0750am-0805am/trajectories-0750am-0805am.txt']


datasetV2Test=tensorsDatasetV2Test(trajectoryFileList)

print(datasetV2Test.getNormalizationDict())

import os
firstCombined,secondCombined=datasetV2Test.__getitem__(40)
print(firstCombined.shape,secondCombined.shape)
image=visualizeTensorData(firstCombined[0,:],firstCombined[1,:],normalizationDict=datasetV2Test.getNormalizationDict())
dirName='combinedTensorFolder'+str(int(time.time()))
os.mkdir(dirName)
for i in range(0,2000):
    fileName='./'+dirName+'/'+str(10000000+i)+'.png'
    firstCombined,secondCombined=datasetV2Test.__getitem__(i)
    image=visualizeTensorData(firstCombined[0,:],firstCombined[1,:],normalizationDict=datasetV2Test.getNormalizationDict())
    cv2.imwrite(fileName,image)

In [29]:
import math
class tensorsDatasetV3(Dataset):
    """
    dataset for relation lstm model
    """
    def __init__(self, trajectoryFileList, numberOfTensorsEachBatch=10,lableTensorEachBatch=10):

        self.valueDictList=[]
        self.sizeList=[]
        self.numberOfTensorsEachBatch=numberOfTensorsEachBatch
        self.lableTensorEachBatch=lableTensorEachBatch
        self.allTensorsEachBatch=numberOfTensorsEachBatch+lableTensorEachBatch
        self.keysList=[]
        if doNormalization:
            self.positionXMax=0
            self.positionXMin=999999
            self.positionYMax=0
            self.positionYMin=99999
            self.speedMax=-100
            self.speedMin=999999
            self.accMax=-100
            self.accMin=9999
        for eachFile in trajectoryFileList:
            valueDict=fromDirGenerateDict(eachFile)
            if doNormalization:
                #get the max and min value for normalization
                maxAndMinDict=getMaxAndMinValueFromValueDict(valueDict,['Local_X','Local_Y','v_Vel','v_Acc'])
                #position X
                if self.positionXMax<maxAndMinDict['Local_X']['max']:
                    self.positionXMax=maxAndMinDict['Local_X']['max']
                if self.positionXMin>maxAndMinDict['Local_X']['min']:
                    self.positionXMin=maxAndMinDict['Local_X']['min']
                #position Y
                if self.positionYMax<maxAndMinDict['Local_Y']['max']:
                    self.positionYMax=maxAndMinDict['Local_Y']['max']
                if self.positionYMin>maxAndMinDict['Local_Y']['min']:
                    self.positionYMin=maxAndMinDict['Local_Y']['min']
                #speed
                if self.speedMax<maxAndMinDict['v_Vel']['max']:
                    self.speedMax=maxAndMinDict['v_Vel']['max']
                if self.speedMin>maxAndMinDict['v_Vel']['min']:
                    self.speedMin=maxAndMinDict['v_Vel']['min']
                #acc
                if self.accMax<maxAndMinDict['v_Acc']['max']:
                    self.accMax=maxAndMinDict['v_Acc']['max']
                if self.accMin>maxAndMinDict['v_Acc']['min']:
                    self.accMin=maxAndMinDict['v_Acc']['min']
            self.valueDictList.append(copy.deepcopy(valueDict))
            self.sizeList.append(valueDict.keys().__len__()-self.allTensorsEachBatch)
            sortedKeys=list(valueDict.keys())
            sortedKeys.sort()
            self.keysList.append(copy.deepcopy(sortedKeys))

    def getNormalizationDict(self):
        '''
        Returns:
            a dict:{'positionXMax':self.positionXMax,'positonYMax':self.self.positionYMax,\
               'positionXMin':self.positionXMin,'positionYMin':self.positionYMin,\
               'speedMax':self.speedMax,'speedMin':speedMin,\
               'accMax':self.accMax,'accMax':self.accMin}
        '''
        if not doNormalization:
            raise Exception('NORMALIZATION IS NOT APPLIED')
        return {'positionXMax':self.positionXMax,'positionYMax':self.positionYMax,\
               'positionXMin':self.positionXMin,'positionYMin':self.positionYMin,\
               'speedMax':self.speedMax,'speedMin':self.speedMin,\
               'accMax':self.accMax,'accMax':self.accMin}
    
    def __len__(self):
        allLen=0
        for length in self.sizeList:
            allLen=allLen+length
        return allLen
    
    def fromIdxMapToList(self,idx):
        """
        since there are several lists,we have to know which list shoud we use and the true index in the list
        Return:
            the trueIndex, listIndex
        """
        countSection=0
        for i in range(0,self.sizeList.__len__()):
            countSection+=self.sizeList[i]
            if(idx<countSection):
                return idx-countSection+self.sizeList[i],i
            
    
            
    
    def __getitem__(self,idx):
        #initialize trueIndex, listIndex
        #
        #this dict record the relation between matrix index and vehicle ID in the last frame
        matrixIndexAndVehicleIDRecordDict={}
        #initialize dict above
        for i in range(0,maxMatrixIndex):
            matrixIndexAndVehicleIDRecordDict[i]={'Vehicle_ID':-1,'refresh':-1}
        matrixIndexAndVehicleIDRecordDict['time']=-1
        trueIndex,listIndex=self.fromIdxMapToList(idx)
        itemDict={'positionTensorList':[],'speedTensorList':[],'accTensorList':[],'angleTensorList':[],'time':[]}
        valueDict=self.valueDictList[listIndex] #valueDict is the Dict of many frames
        dictKeys=self.keysList[listIndex]
        #generate tensors of first frame
        positionTensor,speedTensor,accTensor,angleTensor,newVehicleList\
        =readFirstFrame(matrixIndexAndVehicleIDRecordDict,valueDict[dictKeys[trueIndex]])
        angleSinTensor=torch.sin(angleTensor)
        angleCosTensor=torch.cos(angleTensor)
        #first frame normalization
        if doNormalization:
#             print('before nomalization')
#             print(positionTensor.shape,speedTensor.shape,accTensor.shape)
            positionTensor=torch.cat((torch.div(torch.sub(positionTensor[0,:],self.positionXMin),self.positionXMax-self.positionXMin).unsqueeze(0),\
                                     torch.div(torch.sub(positionTensor[1,:],self.positionYMin,),self.positionYMax-self.positionYMin).unsqueeze(0)),0)
            speedTensor=torch.div(torch.sub(speedTensor,self.speedMin),self.speedMax-self.speedMin)
            accTensor=torch.div(torch.sub(accTensor,self.accMin),self.accMax-self.accMin)
#             print('after normalization')
#             print(positionTensor.shape,speedTensor.shape,accTensor.shape)
            combinedTensor=torch.cat((positionTensor,speedTensor.mul(angleSinTensor),\
                                     speedTensor.mul(angleCosTensor),accTensor.mul(angleSinTensor),\
                                     accTensor.mul(angleCosTensor)),0)
        else:
            combinedTensor=torch.cat((positionTensor,speedTensor.mul(angleSinTensor),\
                                     speedTensor.mul(angleCosTensor),accTensor.mul(angleSinTensor),\
                                     accTensor.mul(angleCosTensor)),0)
        
        
        
        allCombineTensorTrain=combinedTensor.unsqueeze(0)
        allCombineTensorValid=0
#         itemDict['positionTensorList'].append(positionTensor)
#         itemDict['speedTensorList'].append(speedTensor)
#         itemDict['accTensorList'].append(accTensor)
#         itemDict['angleTensorList'].append(angleTensor)
#         time=getValueByLable(['Global_Time'],valueDict[dictKeys[trueIndex]][0]) #valueDict[sortedDitKey[theIndexOfSortedDictKey]][0]
#         itemDict['time'].append(time)
        
        for i in range(trueIndex+1,trueIndex+self.allTensorsEachBatch):
            #generate tensor from general frame
            positionTensor,speedTensor,accTensor,angleTensor,newVehicleList,vanishedVehicleList\
            =readGeneralFrame(matrixIndexAndVehicleIDRecordDict,valueDict[dictKeys[i]],positionTensor)
            angleSinTensor=torch.sin(angleTensor)
            angleCosTensor=torch.cos(angleTensor)
            if doNormalization:
                positionTensor=torch.cat((torch.div(torch.sub(positionTensor[0,:],self.positionXMin),self.positionXMax-self.positionXMin).unsqueeze(0),\
                                         torch.div(torch.sub(positionTensor[1,:],self.positionYMin,),self.positionYMax-self.positionYMin).unsqueeze(0)),0)
                speedTensor=torch.div(torch.sub(speedTensor,self.speedMin),self.speedMax-self.speedMin)
                accTensor=torch.div(torch.sub(accTensor,self.accMin),self.accMax-self.accMin)
                combinedTensor=torch.cat((positionTensor,speedTensor.mul(angleSinTensor),\
                                         speedTensor.mul(angleCosTensor),accTensor.mul(angleSinTensor),\
                                         accTensor.mul(angleCosTensor)),0)
            else:
                combinedTensor=torch.cat((positionTensor,speedTensor.mul(angleSinTensor),\
                                     speedTensor.mul(angleCosTensor),accTensor.mul(angleSinTensor),\
                                         accTensor.mul(angleCosTensor)),0)
            if i<self.numberOfTensorsEachBatch+trueIndex:
                allCombineTensorTrain=torch.cat((allCombineTensorTrain,combinedTensor.unsqueeze(0)),0)
            elif i==self.numberOfTensorsEachBatch+trueIndex:
                allCombineTensorValid=combinedTensor.unsqueeze(0)
            else:
                allCombineTensorValid=torch.cat((allCombineTensorValid,combinedTensor.unsqueeze(0)),0)
#             itemDict['positionTensorList'].append(positionTensor)
#             itemDict['speedTensorList'].append(speedTensor)
#             itemDict['accTensorList'].append(accTensor)
#             itemDict['angleTensorList'].append(angleTensor)
#             time=getValueByLable(['Global_Time'],valueDict[dictKeys[i]][0]) #valueDict[sortedDitKey[theIndexOfSortedDictKey]]
#             itemDict['time'].append(time)
        return allCombineTensorTrain,allCombineTensorValid
# trajectoryFileList=['/home/wangyuchen/trajectory_dataset/US101/0750am-0805am/trajectories-0750am-0805am.txt',\
#                    '/home/wangyuchen/trajectory_dataset/US101/0805am-0820am/trajectories-0805am-0820am.txt',\
#                    '/home/wangyuchen/trajectory_dataset/US101/0820am-0835am/trajectories-0820am-0835am.txt']
trajectoryFileList=['/home/wangyuchen/trajectory_dataset/US101/0750am-0805am/trajectories-0750am-0805am.txt']
#run on 2080 in g814
if runOnG814:
    trajectoryFileList=['/home/wangyuchen/US101/0750am-0805am/trajectories-0750am-0805am.txt']


In [30]:
import math
class tensorsDatasetV4(Dataset):
    """
    dataset for difference series type
    """
    def __init__(self, trajectoryFileList, numberOfTensorsEachBatch=10,lableTensorEachBatch=10,\
                maxRelationNumber=20,givenRange=0.08):

        self.valueDictList=[]
        self.sizeList=[]
        self.numberOfTensorsEachBatch=numberOfTensorsEachBatch
        self.lableTensorEachBatch=lableTensorEachBatch
        self.allTensorsEachBatch=numberOfTensorsEachBatch+lableTensorEachBatch
        self.keysList=[]
        if doNormalization:
            self.positionXMax=0
            self.positionXMin=999999
            self.positionYMax=0
            self.positionYMin=99999
            self.speedMax=-100
            self.speedMin=999999
            self.accMax=-100
            self.accMin=9999
        for eachFile in trajectoryFileList:
            valueDict=fromDirGenerateDict(eachFile)
            if doNormalization:
                #get the max and min value for normalization
                maxAndMinDict=getMaxAndMinValueFromValueDict(valueDict,['Local_X','Local_Y','v_Vel','v_Acc'])
                #position X
                if self.positionXMax<maxAndMinDict['Local_X']['max']:
                    self.positionXMax=maxAndMinDict['Local_X']['max']
                if self.positionXMin>maxAndMinDict['Local_X']['min']:
                    self.positionXMin=maxAndMinDict['Local_X']['min']
                #position Y
                if self.positionYMax<maxAndMinDict['Local_Y']['max']:
                    self.positionYMax=maxAndMinDict['Local_Y']['max']
                if self.positionYMin>maxAndMinDict['Local_Y']['min']:
                    self.positionYMin=maxAndMinDict['Local_Y']['min']
                #speed
                if self.speedMax<maxAndMinDict['v_Vel']['max']:
                    self.speedMax=maxAndMinDict['v_Vel']['max']
                if self.speedMin>maxAndMinDict['v_Vel']['min']:
                    self.speedMin=maxAndMinDict['v_Vel']['min']
                #acc
                if self.accMax<maxAndMinDict['v_Acc']['max']:
                    self.accMax=maxAndMinDict['v_Acc']['max']
                if self.accMin>maxAndMinDict['v_Acc']['min']:
                    self.accMin=maxAndMinDict['v_Acc']['min']
            self.valueDictList.append(copy.deepcopy(valueDict))
            self.sizeList.append(valueDict.keys().__len__()-self.allTensorsEachBatch)
            sortedKeys=list(valueDict.keys())
            sortedKeys.sort()
            self.keysList.append(copy.deepcopy(sortedKeys))

    def getNormalizationDict(self):
        '''
        Returns:
            a dict:{'positionXMax':self.positionXMax,'positonYMax':self.self.positionYMax,\
               'positionXMin':self.positionXMin,'positionYMin':self.positionYMin,\
               'speedMax':self.speedMax,'speedMin':speedMin,\
               'accMax':self.accMax,'accMax':self.accMin}
        '''
        if not doNormalization:
            raise Exception('NORMALIZATION IS NOT APPLIED')
        return {'positionXMax':self.positionXMax,'positionYMax':self.positionYMax,\
               'positionXMin':self.positionXMin,'positionYMin':self.positionYMin,\
               'speedMax':self.speedMax,'speedMin':self.speedMin,\
               'accMax':self.accMax,'accMax':self.accMin}
    
    def __len__(self):
        allLen=0
        for length in self.sizeList:
            allLen=allLen+length
        return allLen
    
    def fromIdxMapToList(self,idx):
        """
        since there are several lists,we have to know which list shoud we use and the true index in the list
        Return:
            the trueIndex, listIndex
        """
        countSection=0
        for i in range(0,self.sizeList.__len__()):
            countSection+=self.sizeList[i]
            if(idx<countSection):
                return idx-countSection+self.sizeList[i],i
            
    
            
    
    def __getitem__(self,idx):
        #initialize trueIndex, listIndex
        #
        #this dict record the relation between matrix index and vehicle ID in the last frame
        matrixIndexAndVehicleIDRecordDict={}
        #initialize dict above
        for i in range(0,maxMatrixIndex):
            matrixIndexAndVehicleIDRecordDict[i]={'Vehicle_ID':-1,'refresh':-1}
        matrixIndexAndVehicleIDRecordDict['time']=-1
        trueIndex,listIndex=self.fromIdxMapToList(idx)
        itemDict={'positionTensorList':[],'speedTensorList':[],'accTensorList':[],'angleTensorList':[],'time':[]}
        valueDict=self.valueDictList[listIndex] #valueDict is the Dict of many frames
        dictKeys=self.keysList[listIndex]
        #generate tensors of first frame
        positionTensor,speedTensor,accTensor,angleTensor,newVehicleList\
        =readFirstFrame(matrixIndexAndVehicleIDRecordDict,valueDict[dictKeys[trueIndex]])
        angleSinTensor=torch.sin(angleTensor)
        angleCosTensor=torch.cos(angleTensor)
        #first frame normalization
        if doNormalization:
#             print('before nomalization')
#             print(positionTensor.shape,speedTensor.shape,accTensor.shape)
            positionTensor=torch.cat((torch.div(torch.sub(positionTensor[0,:],self.positionXMin),self.positionXMax-self.positionXMin).unsqueeze(0),\
                                     torch.div(torch.sub(positionTensor[1,:],self.positionYMin,),self.positionYMax-self.positionYMin).unsqueeze(0)),0)
            speedTensor=torch.div(torch.sub(speedTensor,self.speedMin),self.speedMax-self.speedMin)
            accTensor=torch.div(torch.sub(accTensor,self.accMin),self.accMax-self.accMin)
#             print('after normalization')
#             print(positionTensor.shape,speedTensor.shape,accTensor.shape)
            combinedTensor=torch.cat((positionTensor,speedTensor.mul(angleSinTensor),\
                                     speedTensor.mul(angleCosTensor),accTensor.mul(angleSinTensor),\
                                     accTensor.mul(angleCosTensor)),0)
        else:
            combinedTensor=torch.cat((positionTensor,speedTensor.mul(angleSinTensor),\
                                     speedTensor.mul(angleCosTensor),accTensor.mul(angleSinTensor),\
                                     accTensor.mul(angleCosTensor)),0)
        
        
        
        allCombineTensorTrain=combinedTensor.unsqueeze(0)
        allCombineTensorValid=0
#         itemDict['positionTensorList'].append(positionTensor)
#         itemDict['speedTensorList'].append(speedTensor)
#         itemDict['accTensorList'].append(accTensor)
#         itemDict['angleTensorList'].append(angleTensor)
#         time=getValueByLable(['Global_Time'],valueDict[dictKeys[trueIndex]][0]) #valueDict[sortedDitKey[theIndexOfSortedDictKey]][0]
#         itemDict['time'].append(time)
        
        for i in range(trueIndex+1,trueIndex+self.allTensorsEachBatch):
            #generate tensor from general frame
            positionTensor,speedTensor,accTensor,angleTensor,newVehicleList,vanishedVehicleList\
            =readGeneralFrame(matrixIndexAndVehicleIDRecordDict,valueDict[dictKeys[i]],positionTensor)
            angleSinTensor=torch.sin(angleTensor)
            angleCosTensor=torch.cos(angleTensor)
            if doNormalization:
                positionTensor=torch.cat((torch.div(torch.sub(positionTensor[0,:],self.positionXMin),self.positionXMax-self.positionXMin).unsqueeze(0),\
                                         torch.div(torch.sub(positionTensor[1,:],self.positionYMin,),self.positionYMax-self.positionYMin).unsqueeze(0)),0)
                speedTensor=torch.div(torch.sub(speedTensor,self.speedMin),self.speedMax-self.speedMin)
                accTensor=torch.div(torch.sub(accTensor,self.accMin),self.accMax-self.accMin)
                combinedTensor=torch.cat((positionTensor,speedTensor.mul(angleSinTensor),\
                                         speedTensor.mul(angleCosTensor),accTensor.mul(angleSinTensor),\
                                         accTensor.mul(angleCosTensor)),0)
            else:
                combinedTensor=torch.cat((positionTensor,speedTensor.mul(angleSinTensor),\
                                     speedTensor.mul(angleCosTensor),accTensor.mul(angleSinTensor),\
                                         accTensor.mul(angleCosTensor)),0)
            if i<self.numberOfTensorsEachBatch+trueIndex:
                allCombineTensorTrain=torch.cat((allCombineTensorTrain,combinedTensor.unsqueeze(0)),0)
            elif i==self.numberOfTensorsEachBatch+trueIndex:
                allCombineTensorValid=combinedTensor.unsqueeze(0)
            else:
                allCombineTensorValid=torch.cat((allCombineTensorValid,combinedTensor.unsqueeze(0)),0)
#             itemDict['positionTensorList'].append(positionTensor)
#             itemDict['speedTensorList'].append(speedTensor)
#             itemDict['accTensorList'].append(accTensor)
#             itemDict['angleTensorList'].append(angleTensor)
#             time=getValueByLable(['Global_Time'],valueDict[dictKeys[i]][0]) #valueDict[sortedDitKey[theIndexOfSortedDictKey]]
#             itemDict['time'].append(time)
        return allCombineTensorTrain,allCombineTensorValid
# trajectoryFileList=['/home/wangyuchen/trajectory_dataset/US101/0750am-0805am/trajectories-0750am-0805am.txt',\
#                    '/home/wangyuchen/trajectory_dataset/US101/0805am-0820am/trajectories-0805am-0820am.txt',\
#                    '/home/wangyuchen/trajectory_dataset/US101/0820am-0835am/trajectories-0820am-0835am.txt']
trajectoryFileList=['/home/wangyuchen/trajectory_dataset/US101/0750am-0805am/trajectories-0750am-0805am.txt']
#run on 2080 in g814
if runOnG814:
    trajectoryFileList=['/home/wangyuchen/US101/0750am-0805am/trajectories-0750am-0805am.txt']


datasetV2=tensorsDatasetV2(trajectoryFileList)

dataIter=iter(datasetV2)
first,second=dataIter.__next__()
print(first.shape, second.shape)

maxMatrixIndex=250
dataloaderV2=DataLoader(datasetV2,batch_size=4,shuffle=True)
for i,item in enumerate(dataloaderV2):
    if(i>0):
        break
    print(i)
    first,second=item
    print(first.shape,second.shape)
    print(first[0,:5,:6])
    print(first[0,(2,245,246,247,248,249,250,251),6:])

In [31]:
import time
def generateAdjacencyMatrix(batchedPositionTensor,lambdaX,lambdaY,omegaX,omegaY,m):
    """
    Using batched position tensor generate batched adjacency matrix
    Args:
        batchedPositionTensor: a batch of position tensor, which size in (batch, timeSequence,2,vehicles), the 
        value 2 in dim=2 is the position of x and y. 
        lambda1,lambda2,omega1,omega2,m are parameters of the function. m<1
        see detail in my notebook
    Returns:
        a batch of adjacency matrix
    Example:
        if given a batch of combined tensor, named theTensor, which size as below:
            (4,100,6,250)
        which means 4 batches, 100 time step, 6 dimension which respectively of positonx, positony, velocityx, 
        velocityy, accx,accy.
        then we apply the function in such way:
        generateAdjacencyMatrix(theTensor(:,:,0:1,:))
    """
    print(batchedPositionTensor.size())
    sizeOfEachMatrix=batchedPositionTensor[0,0,0,:].size()[0]
    print(sizeOfEachMatrix)
    for batchI in range(batchedPositionTensor.size()[0]): #revolve each batch
#         print('batchI',batchI)
        timeStepsMatrixList=[]
        for timeStepI in range(batchedPositionTensor.size()[1]):#revolve each time step
#             print('timeStepI:',timeStepI)
#             adjacencyMatrix=np.zeros((sizeOfEachMatrix,sizeOfEachMatrix))
            adjacencyList=[]
            tempPositionList=batchedPositionTensor[batchI,timeStepI,:,:].numpy().tolist()
#             start=time.time()
            for i in range(sizeOfEachMatrix):
                tempLineList=[]
                for j in range(sizeOfEachMatrix):
#                     adjacencyMatrix[i,j]=1
                    if (tempPositionList[1][i]*tempPositionList[1][j]==0):
                        toZero=0
                    else:
                        toZero=1
                        
                    #calculate original element with linear function
#                     tempLineList.append((1-abs(tempPositionList[1][i]-tempPositionList[1][j]))*\
#                         (1-abs(tempPositionList[0][i]-tempPositionList[0][j]))*toZero)
                    
                    #calculate original element with exponential function
                    element=(omegaY/(math.exp(lambdaY*(abs(tempPositionList[1][j]-tempPositionList[1][i])))))*\
                    (omegaX/(math.exp(lambdaX*(abs(tempPositionList[0][j]-tempPositionList[0][i])))))*toZero
                    tempLineList.append(element)
#                     adjacencyMatrix[i,j]=(batchedPositionTensor[batchI,timeStepI,1,i]-batchedPositionTensor[batchI,timeStepI,1,j])*\
#                         (batchedPositionTensor[batchI,timeStepI,0,i]-batchedPositionTensor[batchI,timeStepI,0,j])
#                     (omegaY/math.exp(lambdaX*abs(batchedPositionTensor[batchI,timeStepI,1,i]-batchedPositionTensor[batchI,timeStepI,1,j])))*\
#                     (omegaX/math.exp(lambdaY*abs(batchedPositionTensor[batchI,timeStepI,0,i]-batchedPositionTensor[batchI,timeStepI,0,j])))
                    
                    #calculate original element with expenential
#                     adjacencyMatrix[i,j]=
#                     (omegaY/math.exp(lambdaX*abs(batchedPositionTensor[batchI,timeStepI,1,i]-batchedPositionTensor[batchI,timeStepI,1,j])))*\
#                     (omegaX/math.exp(lambdaY*abs(batchedPositionTensor[batchI,timeStepI,0,i]-batchedPositionTensor[batchI,timeStepI,0,j])))
                    if(tempPositionList[1][j]-tempPositionList[1][i]<0):
                        #if i follows j, then multiple m, m<1
                        tempLineList[j]=tempLineList[j]*m
                adjacencyList.append(tempLineList)
            
#             end=time.time()
#             print(end-start)
            adjacencyMatrix=torch.tensor(adjacencyList).unsqueeze(0)
            if timeStepI==0:
                matrixSequenceInTimeStepDim=adjacencyMatrix
            else:
                matrixSequenceInTimeStepDim=\
                torch.cat((matrixSequenceInTimeStepDim,adjacencyMatrix),0)
        matrixSequenceInTimeStepDim=matrixSequenceInTimeStepDim.unsqueeze(0)
        if batchI==0:
            matrixSequenceInBatchDim=matrixSequenceInTimeStepDim
        else:
            matrixSequenceInBatchDim=torch.cat((matrixSequenceInBatchDim,matrixSequenceInTimeStepDim),0)            
    return matrixSequenceInBatchDim

def tensorNormalization(inputTensor,minValue,maxValue):
    inputTensor.div_(maxValue)
    
def batchNormalizationForCombinedTensor(inputBatchedTensor,minX,maxX,minY,maxY,minV,maxV,minA,maxA):
    tensorNormalization(inputBatchedTensor[:,:,0,:],minX,maxX)
    tensorNormalization(inputBatchedTensor[:,:,1,:],minY,maxY)
    tensorNormalization(inputBatchedTensor[:,:,2:4,:],minV,maxV)
    tensorNormalization(inputBatchedTensor[:,:,4:6,:],minA,maxA)

In [30]:
class LSTM(nn.Module):
    """
    __init__(self, input_size, cell_size, hidden_size, output_last = True)
    """
    def __init__(self, input_size, cell_size, hidden_size, output_last = True):
        """
        cell_size is the size of cell_state.
        hidden_size is the size of hidden_state, or say the output_state of each step
        """
        super(LSTM, self).__init__()
        
        self.cell_size = cell_size
        self.hidden_size = hidden_size
        self.fl = nn.Linear(input_size + hidden_size, hidden_size)
        self.il = nn.Linear(input_size + hidden_size, hidden_size)
        self.ol = nn.Linear(input_size + hidden_size, hidden_size)
        self.Cl = nn.Linear(input_size + hidden_size, hidden_size)
        
        self.output_last = output_last
        
    def step(self, input, Hidden_State, Cell_State):
        combined = torch.cat((input, Hidden_State), 1)
        f = F.sigmoid(self.fl(combined))
        i = F.sigmoid(self.il(combined))
        o = F.sigmoid(self.ol(combined))
        C = F.tanh(self.Cl(combined))
        Cell_State = f * Cell_State + i * C
        Hidden_State = o * F.tanh(Cell_State)
        
        return Hidden_State, Cell_State
    
    def forward(self, inputs):
        batch_size = inputs.size(0)
        time_step = inputs.size(1)
        Hidden_State, Cell_State = self.initHidden(batch_size)
        
        if self.output_last:
            for i in range(time_step):
                Hidden_State, Cell_State = self.step(torch.squeeze(inputs[:,i:i+1,:]), Hidden_State, Cell_State)  
            return Hidden_State
        else:
            outputs = None
            for i in range(time_step):
                Hidden_State, Cell_State = self.step(torch.squeeze(inputs[:,i:i+1,:]), Hidden_State, Cell_State)  
                if outputs is None:
                    outputs = Hidden_State.unsqueeze(1)
                else:
                    outputs = torch.cat((outputs, Hidden_State.unsqueeze(1)), 1)
            return outputs
    
    def initHidden(self, batch_size):
        use_gpu = torch.cuda.is_available()
        if use_gpu:
            Hidden_State = Variable(torch.zeros(batch_size, self.hidden_size).cuda())
            Cell_State = Variable(torch.zeros(batch_size, self.hidden_size).cuda())
            return Hidden_State, Cell_State
        else:
            Hidden_State = Variable(torch.zeros(batch_size, self.hidden_size))
            Cell_State = Variable(torch.zeros(batch_size, self.hidden_size))
            return Hidden_State, Cell_State
        

In [31]:
# import math
# maxMatrixIndex=250
# trajectorDataSet=tensorsDataset(trajectoryFileList)


In [32]:
# dataLoader=DataLoader(trajectorDataSet,batch_size=1,shuffle=True,num_workers=4)
# for i,data in enumerate(dataLoader):
#     print('11111')
#     if(i>10):
#         break
#     print(data[0].shape)
#     print(data[0][0,:,1,1:5])

In [33]:
#code fragment below is used to visualize tensor data
# dataLoader=DataLoader(trajectorDataSet,batch_size=1,shuffle=True,num_workers=4)
# for dataI,data in enumerate(dataLoader):
#     if(dataI>10):
#         break
#     for i in range(int(data[0][0,:,0,0].shape[0])):
#         tensorImage=visualizeTensorData(data[0][0,i,0,:],data[0][0,i,1,:],2500,100,10) 
#         fileName=str(100000+dataI)+'_'+str(100000+i)+'.png'
#         cv2.imwrite('./tensorVisualizeFolder/'+fileName,tensorImage)

In [34]:
class relationNetwork(nn.Module):
    """
    process objects to generate relation tensors
    """
    def __init__(self,inputSize=4,outputSize=30,size1=10,size2=10,size3=10,size4=10):
        super(relationNetwork,self).__init__()
        self.size1=size1
        self.size2=size2
        self.size3=size3
        self.size4=size4
        self.layer1=nn.Linear(inputSize,size1)
        self.layer2=nn.Linear(size1,size2)
        self.layer3=nn.Linear(size2,size3)
        self.layer4=nn.Linear(size3,size4)
        self.layer5=nn.Linear(size4,outputSize)
        self.ReLU=nn.ReLU()
        
    def forward(self,inputs):
        x1=self.ReLU(self.layer1(inputs))
        x2=self.ReLU(self.layer2(x1))
        x3=self.ReLU(self.layer3(x2))
        x4=self.ReLU(self.layer4(x3))
        outputs=self.ReLU(self.layer5(x4))
        return outputs
    

In [35]:
class effectNetwork(nn.Module):
    """
    process objects to generate relation tensors
    """
    def __init__(self,inputSize=4,outputSize=30,size1=10,size2=10,size3=10,size4=10):
        super(effectNetwork,self).__init__()
        self.size1=size1
        self.size2=size2
        self.size3=size3
        self.size4=size4
        self.layer1=nn.Linear(inputSize,size1)
        self.layer2=nn.Linear(size1,size2)
        self.layer3=nn.Linear(size2,size3)
        self.layer4=nn.Linear(size3,size4)
        self.layer5=nn.Linear(size4,outputSize)
        self.ReLU=nn.ReLU()
        
    def forward(self,inputs):
        x1=self.ReLU(self.layer1(inputs))
        x2=self.ReLU(self.layer2(x1))
        x3=self.ReLU(self.layer3(x2))
        x4=self.ReLU(self.layer4(x3))
        outputs=self.ReLU(self.layer5(x4))
        return outputs

In [36]:
class effectCombinationNetwork(nn.Module):
    """
    process objects to generate relation tensors
    """
    def __init__(self,inputSize=4,outputSize=30,size1=10,size2=10,size3=10,size4=10):
        super(effectCombinationNetwork,self).__init__()
        self.size1=size1
        self.size2=size2
        self.size3=size3
        self.size4=size4
        self.layer1=nn.Linear(inputSize,size1)
        self.layer2=nn.Linear(size1,size2)
        self.layer3=nn.Linear(size2,size3)
        self.layer4=nn.Linear(size3,size4)
        self.layer5=nn.Linear(size4,outputSize)
        self.ReLU=nn.ReLU()
        
    def forward(self,inputs):
        x1=self.ReLU(self.layer1(inputs))
        x2=self.ReLU(self.layer2(x1))
        x3=self.ReLU(self.layer3(x2))
        x4=self.ReLU(self.layer4(x3))
        outputs=self.ReLU(self.layer5(x4))
        return outputs

In [37]:
class objectModifyNetwork(nn.Module):
    """
    process objects to generate relation tensors
    """
    def __init__(self,inputSize=4,outputSize=30,size1=10,size2=10,size3=10,size4=10):
        super(objectModifyNetwork,self).__init__()
        self.size1=size1
        self.size2=size2
        self.size3=size3
        self.size4=size4
        self.layer1=nn.Linear(inputSize,size1)
        self.layer2=nn.Linear(size1,size2)
        self.layer3=nn.Linear(size2,size3)
        self.layer4=nn.Linear(size3,size4)
        self.layer5=nn.Linear(size4,outputSize)
        self.ReLU=nn.ReLU()
        
    def forward(self,inputs):
        x1=self.ReLU(self.layer1(inputs))
        x2=self.ReLU(self.layer2(x1))
        x3=self.ReLU(self.layer3(x2))
        x4=self.ReLU(self.layer4(x3))
        outputs=self.layer5(x4)
        return outputs

#this block build for relation network testing
#delete later if needed
relationTensorSize=40
positionTuple=(0,1,6,7)
velocityTuple=(2,3,8,9)
acclerateTuple=(4,5,10,11)
positionRelationNet=relationNetwork(outputSize=relationTensorSize)
velocityRelationNet=relationNetwork(outputSize=relationTensorSize)
accelerateRelationNet=relationNetwork(outputSize=relationTensorSize)

maxMatrixIndex=250

#load test data in network testing block
dataloaderV2=DataLoader(datasetV2,batch_size=1,shuffle=True)
for i,item in enumerate(dataloaderV2):
    if(i>0):
        break
    print(i)
    first,second=item
#     print(first.shape,second.shape)
#     print(first[0,:5,:6])
#     print(first[0,(2,245,246,247,248,249,250,251),6:])
    #from frame to position, velocity, accelerate
    positionRelationTensors=positionRelationNet(first[:,:,positionTuple])
    velocityRelationTensors=velocityRelationNet(first[:,:,velocityTuple])
    accelerateRelationTensors=accelerateRelationNet(first[:,:,acclerateTuple])
    print(positionRelationTensors.shape)
    objectsAndRelationTensors=torch.cat((first[:,:,positionTuple],positionRelationTensors),2)
    print(objectsAndRelationTensors.shape)
    

objectAndTensorSize=relationTensorSize+4 
effectOutputTensorSize=20
#the number 4 is is the size of positon(or velocity or accelerate) pairs,
#such as (positonxObject1,positonyObject1,positionxObject2,positionyObject2)
positionEffectNet=effectNetwork(inputSize=objectAndTensorSize,outputSize=effectOutputTensorSize)
velocityEffectNet=effectNetwork(inputSize=objectAndTensorSize,outputSize=effectOutputTensorSize)
accelerateEffectNet=effectNetwork(inputSize=objectAndTensorSize,outputSize=effectOutputTensorSize)
first,second=next(iter(dataloaderV2))
print(first.shape)

#relation computation
positionRelationTensors=positionRelationNet(first[:,:,positionTuple])
velocityRelationTensors=velocityRelationNet(first[:,:,velocityTuple])
accelerateRelationTensors=accelerateRelationNet(first[:,:,acclerateTuple])
                                                      
objectsAndPositionRelationTensors=torch.cat((first[:,:,positionTuple],positionRelationTensors),2)
objectsAndVelocityRelationTensors=torch.cat((first[:,:,positionTuple],velocityRelationTensors),2)
objectsAndAccelerateRelationTensors=torch.cat((first[:,:,positionTuple],accelerateRelationTensors),2)

#effect computation
positionEffectTensors=positionEffectNet(objectsAndPositionRelationTensors)
velocityEffectTensors=velocityEffectNet(objectsAndVelocityRelationTensors)
accelerateEffectTensors=accelerateEffectNet(objectsAndAccelerateRelationTensors)

#effect combination type 1
#tensor summation
batchSize=positionRelationTensors.shape[0]
positionEffectSummation=torch.zeros(batchSize,maxMatrixIndex,effectOutputTensorSize)
velocityEffectSummation=torch.zeros(batchSize,maxMatrixIndex,effectOutputTensorSize)
accelerateEffectSummation=torch.zeros(batchSize,maxMatrixIndex,effectOutputTensorSize)
for i in range(maxMatrixIndex):
    positionEffectSummation[:,i,:]=torch.sum(positionEffectTensors[:,(i*249):((i+1)*249),:],1)
    velocityEffectSummation[:,i,:]=torch.sum(velocityEffectTensors[:,(i*249):((i+1)*249),:],1)
    accelerateEffectSummation[:,i,:]=torch.sum(accelerateEffectTensors[:,(i*249):((i+1)*249),:],1)
    
#effect combination net initial
#combined tensors length
combinedTensorSize=20
effectCombinationNet=effectCombinationNetwork(inputSize=effectOutputTensorSize*3,outputSize=combinedTensorSize)

#combine tensors and process the combined one
combinedEffectTensors=torch.cat((positionEffectSummation,velocityEffectSummation,accelerateEffectSummation),2)
print(combinedEffectTensors.shape)
processedCombinedEffectTensors=effectCombinationNet(combinedEffectTensors)
print(processedCombinedEffectTensors.shape)

#generate a tuple in which each element is the index of a vehicle
#the tuple is used to get the property of each vehicle from the left side of data from the dataset function
#this part has been put into init function of Module class
listForEachVehicle=[]
for i in range(maxMatrixIndex):
    listForEachVehicle.append(i*(maxMatrixIndex-1))
tupleForEachVehicle=tuple(listForEachVehicle)

#the property of each vehicle
vehicleProperty=first[:,tupleForEachVehicle,0:6]

objectAndFinalEffect=torch.cat((vehicleProperty,processedCombinedEffectTensors),2)
print(objectAndFinalEffect.shape)

#initialize the final network to generate new objects properties
objectModifyNet=objectModifyNetwork(inputSize=combinedTensorSize+6,outputSize=6)
finalObjectState=objectModifyNet(objectAndFinalEffect)
print(finalObjectState.shape)

In [38]:
#from relation to new object network
class fromRelationToObjectNetwork(nn.Module):
    def __init__(self):
        super(fromRelationToObjectNetwork,self).__init__()
        self.maxMatrixIndex=250
        #generate a tuple in which each element is the index of a vehicle
        #the tuple is used to get the property of each vehicle from the left side of data from the dataset function
        listForEachVehicle=[]
        for i in range(self.maxMatrixIndex):
            listForEachVehicle.append(i*(self.maxMatrixIndex-1))
        self.tupleForEachVehicle=tuple(listForEachVehicle)
        
        #position relation network initialize
        self.relationTensorSize=40
        self.positionTuple=(0,1,6,7)
        self.velocityTuple=(2,3,8,9)
        self.acclerateTuple=(4,5,10,11)
        self.positionRelationNet=relationNetwork(outputSize=self.relationTensorSize)
        self.velocityRelationNet=relationNetwork(outputSize=self.relationTensorSize)
        self.accelerateRelationNet=relationNetwork(outputSize=self.relationTensorSize)
        
        #effect network initialize
        self.objectAndTensorSize=self.relationTensorSize+4 
        self.effectOutputTensorSize=20
        #the number 4 is is the size of positon(or velocity or accelerate) pairs,
        #such as (positonxObject1,positonyObject1,positionxObject2,positionyObject2)
        self.positionEffectNet=effectNetwork(inputSize=self.objectAndTensorSize,outputSize=self.effectOutputTensorSize)
        self.velocityEffectNet=effectNetwork(inputSize=self.objectAndTensorSize,outputSize=self.effectOutputTensorSize)
        self.accelerateEffectNet=effectNetwork(inputSize=self.objectAndTensorSize,outputSize=self.effectOutputTensorSize)
        
        #effect combination net initialize
        #combined tensors length
        self.combinedTensorSize=20
        self.effectCombinationNet=\
        effectCombinationNetwork(inputSize=self.effectOutputTensorSize*3,outputSize=self.combinedTensorSize)
        
        #initialize the final network to generate new objects properties
        self.objectModifyNet=objectModifyNetwork(inputSize=self.combinedTensorSize+6,outputSize=6)
        
    def forward(self,inputObjectsPairs):
        #relation computation
        positionRelationTensors=self.positionRelationNet(inputObjectsPairs[:,:,self.positionTuple])
        velocityRelationTensors=self.velocityRelationNet(inputObjectsPairs[:,:,self.velocityTuple])
        accelerateRelationTensors=self.accelerateRelationNet(inputObjectsPairs[:,:,self.acclerateTuple])

        objectsAndPositionRelationTensors=torch.cat((inputObjectsPairs[:,:,self.positionTuple],positionRelationTensors),2)
        objectsAndVelocityRelationTensors=torch.cat((inputObjectsPairs[:,:,self.velocityTuple],velocityRelationTensors),2)
        objectsAndAccelerateRelationTensors=torch.cat((inputObjectsPairs[:,:,self.acclerateTuple],accelerateRelationTensors),2)

        #effect computation
        positionEffectTensors=self.positionEffectNet(objectsAndPositionRelationTensors)
        velocityEffectTensors=self.velocityEffectNet(objectsAndVelocityRelationTensors)
        accelerateEffectTensors=self.accelerateEffectNet(objectsAndAccelerateRelationTensors)

        
        #effect combination type 1
        #tensor summation
        batchSize=positionRelationTensors.shape[0]
        if useGpu==True:
            positionEffectSummation=torch.zeros(batchSize,maxMatrixIndex,effectOutputTensorSize).cuda()
            velocityEffectSummation=torch.zeros(batchSize,maxMatrixIndex,effectOutputTensorSize).cuda()
            accelerateEffectSummation=torch.zeros(batchSize,maxMatrixIndex,effectOutputTensorSize).cuda()
        else: 
            positionEffectSummation=torch.zeros(batchSize,maxMatrixIndex,effectOutputTensorSize)
            velocityEffectSummation=torch.zeros(batchSize,maxMatrixIndex,effectOutputTensorSize)
            accelerateEffectSummation=torch.zeros(batchSize,maxMatrixIndex,effectOutputTensorSize)
        for i in range(maxMatrixIndex):
            positionEffectSummation[:,i,:]=torch.sum(positionEffectTensors[:,(i*249):((i+1)*249),:],1)
            velocityEffectSummation[:,i,:]=torch.sum(velocityEffectTensors[:,(i*249):((i+1)*249),:],1)
            accelerateEffectSummation[:,i,:]=torch.sum(accelerateEffectTensors[:,(i*249):((i+1)*249),:],1)
        combinedEffectTensors=torch.cat((positionEffectSummation,velocityEffectSummation,accelerateEffectSummation),2)
        processedCombinedEffectTensors=self.effectCombinationNet(combinedEffectTensors)
        
        #the property of each vehicle
        vehicleProperty=inputObjectsPairs[:,self.tupleForEachVehicle,0:6]
        objectAndFinalEffect=torch.cat((vehicleProperty,processedCombinedEffectTensors),2)
        
        #compute final state
        finalObjectState=self.objectModifyNet(objectAndFinalEffect)
        return finalObjectState

In [39]:
#This module is supposed to be placed in LSTM model
class fromRelationToEffectNetwork(nn.Module):
    def __init__(self,effectOutputTensorSize=20):
        super(fromRelationToEffectNetwork,self).__init__()
        self.maxMatrixIndex=250
        #generate a tuple in which each element is the index of a vehicle
        #the tuple is used to get the property of each vehicle from the left side of data from the dataset function
        listForEachVehicle=[]
        for i in range(self.maxMatrixIndex):
            listForEachVehicle.append(i*(self.maxMatrixIndex-1))
        self.tupleForEachVehicle=tuple(listForEachVehicle)
        
        #position relation network initialize
        self.relationTensorSize=40
        self.positionTuple=(0,1,6,7)
        self.velocityTuple=(2,3,8,9)
        self.acclerateTuple=(4,5,10,11)
        self.positionRelationNet=relationNetwork(outputSize=self.relationTensorSize)
        self.velocityRelationNet=relationNetwork(outputSize=self.relationTensorSize)
        self.accelerateRelationNet=relationNetwork(outputSize=self.relationTensorSize)
        
        #effect network initialize
        self.objectAndTensorSize=self.relationTensorSize+4 
        self.effectOutputTensorSize=effectOutputTensorSize
        #the number 4 is the size of positon(or velocity or accelerate) pairs,
        #such as (positonxObject1,positonyObject1,positionxObject2,positionyObject2)
        self.positionEffectNet=effectNetwork(inputSize=self.objectAndTensorSize,outputSize=self.effectOutputTensorSize)
        self.velocityEffectNet=effectNetwork(inputSize=self.objectAndTensorSize,outputSize=self.effectOutputTensorSize)
        self.accelerateEffectNet=effectNetwork(inputSize=self.objectAndTensorSize,outputSize=self.effectOutputTensorSize)
        
        #effect combination net initialize
        #combined tensors length
        self.combinedTensorSize=20
        self.effectCombinationNet=\
        effectCombinationNetwork(inputSize=self.effectOutputTensorSize*3,outputSize=self.combinedTensorSize)
        
        #remove objectModifyNet in objectToEffectModel
        #initialize the final network to generate new objects properties
#         self.objectModifyNet=objectModifyNetwork(inputSize=self.combinedTensorSize+6,outputSize=6)
        
    def forward(self,inputObjectsPairs):
        #relation computation
        effectOutputTensorSize=self.effectOutputTensorSize
        positionRelationTensors=self.positionRelationNet(inputObjectsPairs[:,:,self.positionTuple])
        velocityRelationTensors=self.velocityRelationNet(inputObjectsPairs[:,:,self.velocityTuple])
        accelerateRelationTensors=self.accelerateRelationNet(inputObjectsPairs[:,:,self.acclerateTuple])

        objectsAndPositionRelationTensors=torch.cat((inputObjectsPairs[:,:,self.positionTuple],positionRelationTensors),2)
        objectsAndVelocityRelationTensors=torch.cat((inputObjectsPairs[:,:,self.velocityTuple],velocityRelationTensors),2)
        objectsAndAccelerateRelationTensors=torch.cat((inputObjectsPairs[:,:,self.acclerateTuple],accelerateRelationTensors),2)

        #effect computation
        positionEffectTensors=self.positionEffectNet(objectsAndPositionRelationTensors)
        velocityEffectTensors=self.velocityEffectNet(objectsAndVelocityRelationTensors)
        accelerateEffectTensors=self.accelerateEffectNet(objectsAndAccelerateRelationTensors)

        
        #effect combination type 1
        #tensor summation
        batchSize=positionRelationTensors.shape[0]
        if useGpu==True:
            positionEffectSummation=torch.zeros(batchSize,maxMatrixIndex,effectOutputTensorSize).cuda()
            velocityEffectSummation=torch.zeros(batchSize,maxMatrixIndex,effectOutputTensorSize).cuda()
            accelerateEffectSummation=torch.zeros(batchSize,maxMatrixIndex,effectOutputTensorSize).cuda()
        else: 
            positionEffectSummation=torch.zeros(batchSize,maxMatrixIndex,effectOutputTensorSize)
            velocityEffectSummation=torch.zeros(batchSize,maxMatrixIndex,effectOutputTensorSize)
            accelerateEffectSummation=torch.zeros(batchSize,maxMatrixIndex,effectOutputTensorSize)
        for i in range(maxMatrixIndex):
            positionEffectSummation[:,i,:]=torch.sum(positionEffectTensors[:,(i*249):((i+1)*249),:],1)
            velocityEffectSummation[:,i,:]=torch.sum(velocityEffectTensors[:,(i*249):((i+1)*249),:],1)
            accelerateEffectSummation[:,i,:]=torch.sum(accelerateEffectTensors[:,(i*249):((i+1)*249),:],1)
        combinedEffectTensors=torch.cat((positionEffectSummation,velocityEffectSummation,accelerateEffectSummation),2)
        processedCombinedEffectTensors=self.effectCombinationNet(combinedEffectTensors)
        
        #remove object extraction component and computation component
#         #the property of each vehicle
#         vehicleProperty=inputObjectsPairs[:,self.tupleForEachVehicle,0:6]
#         objectAndFinalEffect=torch.cat((vehicleProperty,processedCombinedEffectTensors),2)
        
#         #compute final state
#         finalObjectState=self.objectModifyNet(objectAndFinalEffect)
        return processedCombinedEffectTensors

In [40]:
#This module is supposed to be placed in LSTM model
class fromRelationToEffectNetworkPositionOnly(nn.Module):
    def __init__(self,effectOutputTensorSize=20):
        super(fromRelationToEffectNetwork,self).__init__()
        self.maxMatrixIndex=250
        #generate a tuple in which each element is the index of a vehicle
        #the tuple is used to get the property of each vehicle from the left side of data from the dataset function
        listForEachVehicle=[]
        for i in range(self.maxMatrixIndex):
            listForEachVehicle.append(i*(self.maxMatrixIndex-1))
        self.tupleForEachVehicle=tuple(listForEachVehicle)
        
        #position relation network initialize
        self.relationTensorSize=40
        self.positionTuple=(0,1,6,7)
#         self.velocityTuple=(2,3,8,9)
#         self.acclerateTuple=(4,5,10,11)
        self.positionRelationNet=relationNetwork(outputSize=self.relationTensorSize)
#         self.velocityRelationNet=relationNetwork(outputSize=self.relationTensorSize)
#         self.accelerateRelationNet=relationNetwork(outputSize=self.relationTensorSize)
        
        #effect network initialize
        self.objectAndTensorSize=self.relationTensorSize+4 
        self.effectOutputTensorSize=effectOutputTensorSize
        #the number 4 is the size of positon(or velocity or accelerate) pairs,
        #such as (positonxObject1,positonyObject1,positionxObject2,positionyObject2)
        self.positionEffectNet=effectNetwork(inputSize=self.objectAndTensorSize,outputSize=self.effectOutputTensorSize)
#         self.velocityEffectNet=effectNetwork(inputSize=self.objectAndTensorSize,outputSize=self.effectOutputTensorSize)
#         self.accelerateEffectNet=effectNetwork(inputSize=self.objectAndTensorSize,outputSize=self.effectOutputTensorSize)
        
        #effect combination net initialize
        #combined tensors length
#         self.combinedTensorSize=20
#         self.effectCombinationNet=\
#         effectCombinationNetwork(inputSize=self.effectOutputTensorSize*3,outputSize=self.combinedTensorSize)
        
        #remove objectModifyNet in objectToEffectModel
        #initialize the final network to generate new objects properties
#         self.objectModifyNet=objectModifyNetwork(inputSize=self.combinedTensorSize+6,outputSize=6)
        
    def forward(self,inputObjectsPairs):
        #relation computation
        effectOutputTensorSize=self.effectOutputTensorSize
        positionRelationTensors=self.positionRelationNet(inputObjectsPairs[:,:,self.positionTuple])
#         velocityRelationTensors=self.velocityRelationNet(inputObjectsPairs[:,:,self.velocityTuple])
#         accelerateRelationTensors=self.accelerateRelationNet(inputObjectsPairs[:,:,self.acclerateTuple])

        objectsAndPositionRelationTensors=torch.cat((inputObjectsPairs[:,:,self.positionTuple],positionRelationTensors),2)
#         objectsAndVelocityRelationTensors=torch.cat((inputObjectsPairs[:,:,self.velocityTuple],velocityRelationTensors),2)
#         objectsAndAccelerateRelationTensors=torch.cat((inputObjectsPairs[:,:,self.acclerateTuple],accelerateRelationTensors),2)

        #effect computation
        positionEffectTensors=self.positionEffectNet(objectsAndPositionRelationTensors)
#         velocityEffectTensors=self.velocityEffectNet(objectsAndVelocityRelationTensors)
#         accelerateEffectTensors=self.accelerateEffectNet(objectsAndAccelerateRelationTensors)

        
        #effect combination type 1
        #tensor summation
        batchSize=positionRelationTensors.shape[0]
        if useGpu==True:
            positionEffectSummation=torch.zeros(batchSize,maxMatrixIndex,effectOutputTensorSize).cuda()
#             velocityEffectSummation=torch.zeros(batchSize,maxMatrixIndex,effectOutputTensorSize).cuda()
#             accelerateEffectSummation=torch.zeros(batchSize,maxMatrixIndex,effectOutputTensorSize).cuda()
        else: 
            positionEffectSummation=torch.zeros(batchSize,maxMatrixIndex,effectOutputTensorSize)
#             velocityEffectSummation=torch.zeros(batchSize,maxMatrixIndex,effectOutputTensorSize)
#             accelerateEffectSummation=torch.zeros(batchSize,maxMatrixIndex,effectOutputTensorSize)
        for i in range(maxMatrixIndex):
            positionEffectSummation[:,i,:]=torch.sum(positionEffectTensors[:,(i*249):((i+1)*249),:],1)
#             velocityEffectSummation[:,i,:]=torch.sum(velocityEffectTensors[:,(i*249):((i+1)*249),:],1)
#             accelerateEffectSummation[:,i,:]=torch.sum(accelerateEffectTensors[:,(i*249):((i+1)*249),:],1)
#         combinedEffectTensors=torch.cat((positionEffectSummation,velocityEffectSummation,accelerateEffectSummation),2)
#         processedCombinedEffectTensors=self.effectCombinationNet(combinedEffectTensors)
        
        #remove object extraction component and computation component
#         #the property of each vehicle
#         vehicleProperty=inputObjectsPairs[:,self.tupleForEachVehicle,0:6]
#         objectAndFinalEffect=torch.cat((vehicleProperty,processedCombinedEffectTensors),2)
        
#         #compute final state
#         finalObjectState=self.objectModifyNet(objectAndFinalEffect)
        return positionEffectSummation

In [41]:
class RelationLSTM(nn.Module):
    def __init__(self, input_size=20, cell_size=20, hidden_size=20, output_last = True):
        """
        cell_size is the size of cell_state.
        hidden_size is the size of hidden_state, or say the output_state of each step
        the input of LSTM  structure was the output of 'fromRelationToEffectNet' module, so that 
        the effectOutputTensorSize has the same number as input_size. 
        
        """
        super(RelationLSTM, self).__init__()
        self.maxMatrixIndex=250
        
        #generate a tuple in which each element is the index of a vehicle
        #the tuple is used to get the property of each vehicle from the left side of data from the dataset function
        listForEachVehicle=[]
        for i in range(self.maxMatrixIndex):
            listForEachVehicle.append(i*(self.maxMatrixIndex-1))
        self.tupleForEachVehicle=tuple(listForEachVehicle)
        self.positionTuple=(0,1,6,7)
        self.velocityTuple=(2,3,8,9)
        self.acclerateTuple=(4,5,10,11)
        
        self.cell_size = cell_size
        self.hidden_size = hidden_size
        self.fl = nn.Linear(input_size + hidden_size, hidden_size)
        self.il = nn.Linear(input_size + hidden_size, hidden_size)
        self.ol = nn.Linear(input_size + hidden_size, hidden_size)
        self.Cl = nn.Linear(input_size + hidden_size, hidden_size)
        self.fromRelationToEffectNet=fromRelationToEffectNetwork(effectOutputTensorSize=input_size)
        self.objectModifyNet=objectModifyNetwork(inputSize=hidden_size+6,outputSize=6)
        
        self.output_last = output_last
        
    def step(self, inputEffectTensor, Hidden_State, Cell_State):
        combined = torch.cat((inputEffectTensor, Hidden_State), 2)
        print('in step,combined.shape',combined.shape)
        f = F.sigmoid(self.fl(combined))
        i = F.sigmoid(self.il(combined))
        o = F.sigmoid(self.ol(combined))
        C = F.tanh(self.Cl(combined))
        Cell_State = f * Cell_State + i * C
        Hidden_State = o * F.tanh(Cell_State)
        
        return Hidden_State, Cell_State
    
    def forward(self, inputs):
        batch_size = inputs.size(0)
        time_step = inputs.size(1)
        Hidden_State, Cell_State = self.initHidden(batch_size)
        
        if self.output_last:
            for i in range(time_step):
                effects=self.fromRelationToEffectNet(inputs[:,i,:,:])
                Hidden_State, Cell_State = self.step(torch.squeeze(effects), Hidden_State, Cell_State) 
                #the property of each vehicle
            print(inputs.shape)
            vehicleProperty=inputs[:,-1,self.tupleForEachVehicle,0:6].squeeze()
            objectAndFinalEffect=torch.cat((vehicleProperty,Hidden_State),2)
            outputState=self.objectModifyNet(objectAndFinalEffect)
            return outputState
        else:
            outputs = None
            for i in range(time_step):
                Hidden_State, Cell_State = self.step(torch.squeeze(inputs[:,i:i+1,:,:]), Hidden_State, Cell_State)  
                if outputs is None:
                    outputs = Hidden_State.unsqueeze(1)
                else:
                    outputs = torch.cat((outputs, Hidden_State.unsqueeze(1)), 1)
            return outputs
    
    def initHidden(self, batch_size):
#         use_gpu = torch.cuda.is_available()
        if useGpu:
            Hidden_State = Variable(torch.zeros(batch_size,maxMatrixIndex, self.hidden_size).cuda())
            Cell_State = Variable(torch.zeros(batch_size,maxMatrixIndex, self.hidden_size).cuda())
            return Hidden_State, Cell_State
        else:
            Hidden_State = Variable(torch.zeros(batch_size, maxMatrixIndex,self.hidden_size))
            Cell_State = Variable(torch.zeros(batch_size, maxMatrixIndex,self.hidden_size))
            return Hidden_State, Cell_State
        

In [42]:
import random
def fromObjectsToRelationPairs(combinedTensor):
    '''
    Args:
        The input tensor should already be transposed if it is generated from the network's output
    Returns:
        Relation pairs
    '''
    #generate relation tensor for all vehicle pairs
    relationTensorLeft=combinedTensor[:,0].expand(combinedTensor.shape[1]-1,combinedTensor.shape[0])
    relationTensorRight=torch.transpose(torch.cat((combinedTensor[:,:0],combinedTensor[:,1:]),1),0,1)
#         print(relationTensorRight.shape,relationTensorRight.shape)
    for i in range(1,combinedTensor.shape[1]):
        relationTensorLeft=torch.cat((relationTensorLeft,\
                                      combinedTensor[:,i].expand(combinedTensor.shape[1]-1,combinedTensor.shape[0])),0)
        relationTensorRight=torch.cat((relationTensorRight,\
                                       torch.transpose(torch.cat((combinedTensor[:,:i],combinedTensor[:,i+1:]),1),0,1)),0)
#         print(relationTensorLeft.shape,relationTensorRight.shape)
    combinedRelationTensor=torch.cat((relationTensorLeft,relationTensorRight),1) 
    return combinedRelationTensor



In [43]:
class RelationLSTMSeq2Seq(nn.Module):
    def __init__(self, input_size=20, cell_size=20, hidden_size=20, \
                 input_size_2=20, cell_size_2=20,hidden_size_2=20, outputTimeFrame=5,output_last = True):
        """
        cell_size is the size of cell_state.
        hidden_size is the size of hidden_state, or say the output_state of each step
        the input of LSTM  structure was the output of 'fromRelationToEffectNet' module, so that 
        the effectOutputTensorSize has the same number as input_size. 
        
        """
        super(RelationLSTMSeq2Seq, self).__init__()
        self.maxMatrixIndex=250
        
        #generate a tuple in which each element is the index of a vehicle
        #the tuple is used to get the property of each vehicle from the left side of data from the dataset function
        listForEachVehicle=[]
        for i in range(self.maxMatrixIndex):
            listForEachVehicle.append(i*(self.maxMatrixIndex-1))
        self.tupleForEachVehicle=tuple(listForEachVehicle)
        self.positionTuple=(0,1,6,7)
        self.velocityTuple=(2,3,8,9)
        self.acclerateTuple=(4,5,10,11)
        
        #effect representive vector computation lstm
        self.cell_size = cell_size
        self.hidden_size = hidden_size
        self.fl = nn.Linear(input_size + hidden_size, hidden_size)
        self.il = nn.Linear(input_size + hidden_size, hidden_size)
        self.ol = nn.Linear(input_size + hidden_size, hidden_size)
        self.Cl = nn.Linear(input_size + hidden_size, hidden_size)
        
        #object modifying lstm
        self.cell_size2 = cell_size_2
        self.hidden_size2 = hidden_size_2
        self.fl2 = nn.Linear(input_size_2 + hidden_size_2, hidden_size_2)
        self.il2 = nn.Linear(input_size_2 + hidden_size_2, hidden_size_2)
        self.ol2 = nn.Linear(input_size_2 + hidden_size_2, hidden_size_2)
        self.Cl2 = nn.Linear(input_size_2 + hidden_size_2, hidden_size_2)
        
        self.outputTimeFrame=outputTimeFrame

        self.fromRelationToEffectNet=fromRelationToEffectNetwork(effectOutputTensorSize=input_size)
        self.objectModifyNet=objectModifyNetwork(inputSize=hidden_size+6,outputSize=6)
        
        self.output_last = output_last
        
    def step(self, inputEffectTensor, Hidden_State, Cell_State):
        combined = torch.cat((inputEffectTensor, Hidden_State), 2)
#         print('in step,combined.shape',combined.shape)
        f = F.sigmoid(self.fl(combined))
        i = F.sigmoid(self.il(combined))
        o = F.sigmoid(self.ol(combined))
        C = F.tanh(self.Cl(combined))
        Cell_State = f * Cell_State + i * C
        Hidden_State = o * F.tanh(Cell_State)
        
        return Hidden_State, Cell_State
    
    def step2(self, inputPreEffectTensor, Hidden_State, Cell_State):
        combined = torch.cat((inputPreEffectTensor, Hidden_State), 2)
#         print('in step2,combined.shape',combined.shape)
        f = F.sigmoid(self.fl2(combined))
        i = F.sigmoid(self.il2(combined))
        o = F.sigmoid(self.ol2(combined))
        C = F.tanh(self.Cl2(combined))
        Cell_State = f * Cell_State + i * C
        Hidden_State = o * F.tanh(Cell_State)
        
        return Hidden_State, Cell_State
    
    def forward(self, inputs):
        batch_size = inputs.size(0)
        time_step = inputs.size(1)
        Hidden_State, Cell_State,Hidden_State_2,Cell_State_2 = self.initHidden(batch_size)
        
        if self.output_last:
            #effect computation
            for i in range(time_step):
                effects=self.fromRelationToEffectNet(inputs[:,i,:,:])
                Hidden_State, Cell_State = self.step(torch.squeeze(effects), Hidden_State, Cell_State) 
                #the property of each vehicle
                
            Hidden_State_2=Hidden_State
            Cell_State_2=Cell_State
            
            #the lstm process below take as inputs the previous object states and output the next predicted states
            #applying function permute to deal with the dimension inconsistency
            '''
            inputs should be processed by function "fromObjectsToRelationPairsBatchAndTimestepVersion" 
            IF DATASETV3 VERSION IS USED TO GET DATA
            '''
            print('in seq2seq, inputs shape:',inputs.shape)
            preVehicleProperty=inputs[:,-1,self.tupleForEachVehicle,0:6].squeeze()
            
            #object properties computation
            for i in range(outputTimeFrame):
                objectAndFinalEffect=torch.cat((preVehicleProperty,Hidden_State_2),2)  
                preVehicleProperty=self.objectModifyNet(objectAndFinalEffect)
                if i==0:
                    #add dimension timestep, which in dimension 1
                    outputVehicleProperties=preVehicleProperty.unsqueeze(1) 
                else:
                    outputVehicleProperties=torch.cat((outputVehicleProperties,preVehicleProperty.unsqueeze(1)),1)
                #don't need to compute new effect vector after the prediction of the last time step
                if i<outputTimeFrame-1:
                    preVehicleProperty.cpu()
                    #add timestep dimension for further process
                    preVehiclePropertyAddTimestep=preVehicleProperty.unsqueeze(1)
                    relationPairs=fromObjectsToRelationPairsBatchAndTimestepVersion(preVehiclePropertyAddTimestep).squeeze()
                    if useGpu:
                        relationPairs.cuda()
                    effectInPropertiesComputation=self.fromRelationToEffectNet(relationPairs)
                    Hidden_State_2,Cell_State_2=self.step2(effectInPropertiesComputation,Hidden_State_2,Cell_State_2)
                if i==outputTimeFrame:
                    break
#             print(inputs.shape)
#             vehicleProperty=inputs[:,-1,self.tupleForEachVehicle,0:6].squeeze()
#             objectAndFinalEffect=torch.cat((vehicleProperty,Hidden_State),2)
#             outputState=self.objectModifyNet(objectAndFinalEffect)
            return outputVehicleProperties
        else:
            outputs = None
            for i in range(time_step):
                Hidden_State, Cell_State = self.step(torch.squeeze(inputs[:,i:i+1,:,:]), Hidden_State, Cell_State)  
                if outputs is None:
                    outputs = Hidden_State.unsqueeze(1)
                else:
                    outputs = torch.cat((outputs, Hidden_State.unsqueeze(1)), 1)
            return outputs
    
    def initHidden(self, batch_size):
#         use_gpu = torch.cuda.is_available()
        if useGpu:
            Hidden_State = Variable(torch.zeros(batch_size,maxMatrixIndex, self.hidden_size).cuda())
            Cell_State = Variable(torch.zeros(batch_size,maxMatrixIndex, self.hidden_size).cuda())
            Hidden_State_2=Variable(torch.zeros(batch_size,maxMatrixIndex,self.hidden_size2).cuda())
            Cell_State_2=Variable(torch.zeros(batch_size,maxMatrixIndex,self.hidden_size2).cuda())
            return Hidden_State, Cell_State,Hidden_State_2,Cell_State_2
        else:
            Hidden_State = Variable(torch.zeros(batch_size, maxMatrixIndex,self.hidden_size))
            Cell_State = Variable(torch.zeros(batch_size, maxMatrixIndex,self.hidden_size))
            Hidden_State_2=Variable(torch.zeros(batch_size,maxMatrixIndex,self.hidden_size2))
            Cell_State_2=Variable(torch.zeros(batch_size,maxMatrixIndex,self.hidden_size2))
            return Hidden_State, Cell_State,Hidden_State_2,Cell_State_2
        

In [44]:
class RelationLSTMSeq2SeqPositionOnly(nn.Module):
    def __init__(self, input_size=20, cell_size=20, hidden_size=20, \
                 input_size_2=20, cell_size_2=20,hidden_size_2=20, outputTimeFrame=5,output_last = True):
        """
        cell_size is the size of cell_state.
        hidden_size is the size of hidden_state, or say the output_state of each step
        the input of LSTM  structure was the output of 'fromRelationToEffectNet' module, so that 
        the effectOutputTensorSize has the same number as input_size. 
        
        """
        super(RelationLSTMSeq2Seq, self).__init__()
        self.maxMatrixIndex=250
        
        #generate a tuple in which each element is the index of a vehicle
        #the tuple is used to get the property of each vehicle from the left side of data from the dataset function
        listForEachVehicle=[]
        for i in range(self.maxMatrixIndex):
            listForEachVehicle.append(i*(self.maxMatrixIndex-1))
        self.tupleForEachVehicle=tuple(listForEachVehicle)
        self.positionTuple=(0,1,6,7)
#         self.velocityTuple=(2,3,8,9)
#         self.acclerateTuple=(4,5,10,11)
        
        #effect representive vector computation lstm
        self.cell_size = cell_size
        self.hidden_size = hidden_size
        self.fl = nn.Linear(input_size + hidden_size, hidden_size)
        self.il = nn.Linear(input_size + hidden_size, hidden_size)
        self.ol = nn.Linear(input_size + hidden_size, hidden_size)
        self.Cl = nn.Linear(input_size + hidden_size, hidden_size)
        
        #object modifying lstm
        self.cell_size2 = cell_size_2
        self.hidden_size2 = hidden_size_2
        self.fl2 = nn.Linear(input_size_2 + hidden_size_2, hidden_size_2)
        self.il2 = nn.Linear(input_size_2 + hidden_size_2, hidden_size_2)
        self.ol2 = nn.Linear(input_size_2 + hidden_size_2, hidden_size_2)
        self.Cl2 = nn.Linear(input_size_2 + hidden_size_2, hidden_size_2)
        
        self.outputTimeFrame=outputTimeFrame

        self.fromRelationToEffectNet=fromRelationToEffectNetworkPositionOnly(effectOutputTensorSize=input_size)
        self.objectModifyNet=objectModifyNetwork(inputSize=hidden_size+6,outputSize=6)
        
        self.output_last = output_last
        
    def step(self, inputEffectTensor, Hidden_State, Cell_State):
        combined = torch.cat((inputEffectTensor, Hidden_State), 2)
#         print('in step,combined.shape',combined.shape)
        f = F.sigmoid(self.fl(combined))
        i = F.sigmoid(self.il(combined))
        o = F.sigmoid(self.ol(combined))
        C = F.tanh(self.Cl(combined))
        Cell_State = f * Cell_State + i * C
        Hidden_State = o * F.tanh(Cell_State)
        
        return Hidden_State, Cell_State
    
    def step2(self, inputPreEffectTensor, Hidden_State, Cell_State):
        combined = torch.cat((inputPreEffectTensor, Hidden_State), 2)
#         print('in step2,combined.shape',combined.shape)
        f = F.sigmoid(self.fl2(combined))
        i = F.sigmoid(self.il2(combined))
        o = F.sigmoid(self.ol2(combined))
        C = F.tanh(self.Cl2(combined))
        Cell_State = f * Cell_State + i * C
        Hidden_State = o * F.tanh(Cell_State)
        
        return Hidden_State, Cell_State
    
    def forward(self, inputs):
        batch_size = inputs.size(0)
        time_step = inputs.size(1)
        Hidden_State, Cell_State,Hidden_State_2,Cell_State_2 = self.initHidden(batch_size)
        
        if self.output_last:
            #effect computation
            for i in range(time_step):
                effects=self.fromRelationToEffectNet(inputs[:,i,:,:])
                Hidden_State, Cell_State = self.step(torch.squeeze(effects), Hidden_State, Cell_State) 
                #the property of each vehicle
                
            Hidden_State_2=Hidden_State
            Cell_State_2=Cell_State
            
            #the lstm process below take as inputs the previous object states and output the next predicted states
            #applying function permute to deal with the dimension inconsistency
            '''
            inputs should be processed by function "fromObjectsToRelationPairsBatchAndTimestepVersion" 
            IF DATASETV3 VERSION IS USED TO GET DATA
            '''
            print('in seq2seq, inputs shape:',inputs.shape)
            preVehicleProperty=inputs[:,-1,self.tupleForEachVehicle,0:6].squeeze()
            
            #object properties computation
            for i in range(outputTimeFrame):
                objectAndFinalEffect=torch.cat((preVehicleProperty,Hidden_State_2),2)  
                preVehicleProperty=self.objectModifyNet(objectAndFinalEffect)
                if i==0:
                    #add dimension timestep, which in dimension 1
                    outputVehicleProperties=preVehicleProperty.unsqueeze(1) 
                else:
                    outputVehicleProperties=torch.cat((outputVehicleProperties,preVehicleProperty.unsqueeze(1)),1)
                #don't need to compute new effect vector after the prediction of the last time step
                if i<outputTimeFrame-1:
                    preVehicleProperty.cpu()
                    #add timestep dimension for further process
                    preVehiclePropertyAddTimestep=preVehicleProperty.unsqueeze(1)
                    relationPairs=fromObjectsToRelationPairsBatchAndTimestepVersion(preVehiclePropertyAddTimestep).squeeze()
                    if useGpu:
                        relationPairs.cuda()
                    effectInPropertiesComputation=self.fromRelationToEffectNet(relationPairs)
                    Hidden_State_2,Cell_State_2=self.step2(effectInPropertiesComputation,Hidden_State_2,Cell_State_2)
                if i==outputTimeFrame:
                    break
#             print(inputs.shape)
#             vehicleProperty=inputs[:,-1,self.tupleForEachVehicle,0:6].squeeze()
#             objectAndFinalEffect=torch.cat((vehicleProperty,Hidden_State),2)
#             outputState=self.objectModifyNet(objectAndFinalEffect)
            return outputVehicleProperties
        else:
            outputs = None
            for i in range(time_step):
                Hidden_State, Cell_State = self.step(torch.squeeze(inputs[:,i:i+1,:,:]), Hidden_State, Cell_State)  
                if outputs is None:
                    outputs = Hidden_State.unsqueeze(1)
                else:
                    outputs = torch.cat((outputs, Hidden_State.unsqueeze(1)), 1)
            return outputs
    
    def initHidden(self, batch_size):
#         use_gpu = torch.cuda.is_available()
        if useGpu:
            Hidden_State = Variable(torch.zeros(batch_size,maxMatrixIndex, self.hidden_size).cuda())
            Cell_State = Variable(torch.zeros(batch_size,maxMatrixIndex, self.hidden_size).cuda())
            Hidden_State_2=Variable(torch.zeros(batch_size,maxMatrixIndex,self.hidden_size2).cuda())
            Cell_State_2=Variable(torch.zeros(batch_size,maxMatrixIndex,self.hidden_size2).cuda())
            return Hidden_State, Cell_State,Hidden_State_2,Cell_State_2
        else:
            Hidden_State = Variable(torch.zeros(batch_size, maxMatrixIndex,self.hidden_size))
            Cell_State = Variable(torch.zeros(batch_size, maxMatrixIndex,self.hidden_size))
            Hidden_State_2=Variable(torch.zeros(batch_size,maxMatrixIndex,self.hidden_size2))
            Cell_State_2=Variable(torch.zeros(batch_size,maxMatrixIndex,self.hidden_size2))
            return Hidden_State, Cell_State,Hidden_State_2,Cell_State_2
        

# custom loss function

In [45]:
class elementWeightedLoss(nn.Module):
    def __init__(self,lambdaDistance,lambdaVelocity,lambdaAccelerate):
        super(elementWeightedLoss,self).__init__()
        self.lambdaDistance,self.lambdaVelocity,self.lambdaAccelerate=\
        lambdaDistance,lambdaVelocity,lambdaAccelerate
        
    def forward(self,output,label):
        '''
        output and label dimension: [batch, timestep, vehiclenum, properties]
        properties dimension:(distancex,distancey,velocityx,velocityy,acceleratex,acceleratey)
        '''
        distanceLoss=torch.mean(torch.pow(output[:,:,:,0:2]-label[:,:,:,0:2],2))
        velocityLoss=torch.mean(torch.pow(output[:,:,:,2:4]-label[:,:,:,2:4],2))
        accelerateLoss=torch.mean(torch.pow(output[:,:,:,4:6]-label[:,:,:,4:6],2))
        return torch.mul(distanceLoss,self.lambdaDistance)+torch.mul(velocityLoss,self.lambdaVelocity)+\
                torch.mul(accelerateLoss,self.lambdaAccelerate)

# TEST RELATION WITHIN A GIVEN RANGE

In [32]:
datasetV3Instance=tensorsDatasetV3(trajectoryFileList,lableTensorEachBatch=5)

In [33]:
datasetV3Loader=DataLoader(datasetV3Instance,batch_size=5,num_workers=4,shuffle=True)

In [67]:
theItemInput,theItemLabel=datasetV3Instance.__getitem__(3000)
print(theItemInput.shape)
# theItemInput=theItemInput.reshape((10,250,6))
# print(theItemInput.shape)

torch.Size([10, 6, 250])


In [83]:
# relationTensor=relationCalculateWithRange(theItemInput[0],0.2,20,20)
# logging.debug(fromAllToStr('relationTensor.shape:  ',relationTensor.shape,'\n','relationTensor: \n',relationTensor))
# discountTensor=discountParameterByExponentialWithDistance(relationTensor)
# logging.debug('discountTensor:'+str(discountTensor))
relationInAllTimeStep,discountInAllTimeStep=computeRelationAndAllTheOtherTensorsWithDistance(theItemInput,theGivenRange=0.15)

2020-08-02 23:49:45,115 -18 - computeRelationAndAllTheOtherTensorsWithDistance - DEBUG - inputFrameTensor.shape:torch.Size([10, 6, 250])
2020-08-02 23:49:45,125 -45 - relationCalculateWithRange - DEBUG - targetVehicleId0
2020-08-02 23:49:45,126 -46 - relationCalculateWithRange - DEBUG - reservedIndexes.__len__()4
2020-08-02 23:49:45,127 -47 - relationCalculateWithRange - DEBUG - propertyTensors[targetVehicleId].shape[0]6
2020-08-02 23:49:45,135 -45 - relationCalculateWithRange - DEBUG - targetVehicleId1
2020-08-02 23:49:45,136 -46 - relationCalculateWithRange - DEBUG - reservedIndexes.__len__()7
2020-08-02 23:49:45,137 -47 - relationCalculateWithRange - DEBUG - propertyTensors[targetVehicleId].shape[0]6
2020-08-02 23:49:45,145 -45 - relationCalculateWithRange - DEBUG - targetVehicleId2
2020-08-02 23:49:45,146 -46 - relationCalculateWithRange - DEBUG - reservedIndexes.__len__()8
2020-08-02 23:49:45,146 -47 - relationCalculateWithRange - DEBUG - propertyTensors[targetVehicleId].shape[0]6

2020-08-02 23:49:45,410 -45 - relationCalculateWithRange - DEBUG - targetVehicleId28
2020-08-02 23:49:45,411 -46 - relationCalculateWithRange - DEBUG - reservedIndexes.__len__()6
2020-08-02 23:49:45,411 -47 - relationCalculateWithRange - DEBUG - propertyTensors[targetVehicleId].shape[0]6
2020-08-02 23:49:45,421 -45 - relationCalculateWithRange - DEBUG - targetVehicleId29
2020-08-02 23:49:45,422 -46 - relationCalculateWithRange - DEBUG - reservedIndexes.__len__()10
2020-08-02 23:49:45,422 -47 - relationCalculateWithRange - DEBUG - propertyTensors[targetVehicleId].shape[0]6
2020-08-02 23:49:45,431 -45 - relationCalculateWithRange - DEBUG - targetVehicleId30
2020-08-02 23:49:45,432 -46 - relationCalculateWithRange - DEBUG - reservedIndexes.__len__()9
2020-08-02 23:49:45,432 -47 - relationCalculateWithRange - DEBUG - propertyTensors[targetVehicleId].shape[0]6
2020-08-02 23:49:45,441 -45 - relationCalculateWithRange - DEBUG - targetVehicleId31
2020-08-02 23:49:45,441 -46 - relationCalculate

2020-08-02 23:49:45,696 -47 - relationCalculateWithRange - DEBUG - propertyTensors[targetVehicleId].shape[0]6
2020-08-02 23:49:45,705 -45 - relationCalculateWithRange - DEBUG - targetVehicleId57
2020-08-02 23:49:45,706 -46 - relationCalculateWithRange - DEBUG - reservedIndexes.__len__()10
2020-08-02 23:49:45,707 -47 - relationCalculateWithRange - DEBUG - propertyTensors[targetVehicleId].shape[0]6
2020-08-02 23:49:45,717 -45 - relationCalculateWithRange - DEBUG - targetVehicleId58
2020-08-02 23:49:45,718 -46 - relationCalculateWithRange - DEBUG - reservedIndexes.__len__()11
2020-08-02 23:49:45,718 -47 - relationCalculateWithRange - DEBUG - propertyTensors[targetVehicleId].shape[0]6
2020-08-02 23:49:45,728 -45 - relationCalculateWithRange - DEBUG - targetVehicleId59
2020-08-02 23:49:45,729 -46 - relationCalculateWithRange - DEBUG - reservedIndexes.__len__()10
2020-08-02 23:49:45,729 -47 - relationCalculateWithRange - DEBUG - propertyTensors[targetVehicleId].shape[0]6
2020-08-02 23:49:45,

2020-08-02 23:49:46,000 -45 - relationCalculateWithRange - DEBUG - targetVehicleId85
2020-08-02 23:49:46,001 -46 - relationCalculateWithRange - DEBUG - reservedIndexes.__len__()11
2020-08-02 23:49:46,001 -47 - relationCalculateWithRange - DEBUG - propertyTensors[targetVehicleId].shape[0]6
2020-08-02 23:49:46,014 -45 - relationCalculateWithRange - DEBUG - targetVehicleId86
2020-08-02 23:49:46,014 -46 - relationCalculateWithRange - DEBUG - reservedIndexes.__len__()12
2020-08-02 23:49:46,015 -47 - relationCalculateWithRange - DEBUG - propertyTensors[targetVehicleId].shape[0]6
2020-08-02 23:49:46,026 -45 - relationCalculateWithRange - DEBUG - targetVehicleId87
2020-08-02 23:49:46,027 -46 - relationCalculateWithRange - DEBUG - reservedIndexes.__len__()12
2020-08-02 23:49:46,027 -47 - relationCalculateWithRange - DEBUG - propertyTensors[targetVehicleId].shape[0]6
2020-08-02 23:49:46,037 -45 - relationCalculateWithRange - DEBUG - targetVehicleId88
2020-08-02 23:49:46,038 -46 - relationCalcula

2020-08-02 23:49:46,354 -46 - relationCalculateWithRange - DEBUG - reservedIndexes.__len__()7
2020-08-02 23:49:46,355 -47 - relationCalculateWithRange - DEBUG - propertyTensors[targetVehicleId].shape[0]6
2020-08-02 23:49:46,369 -45 - relationCalculateWithRange - DEBUG - targetVehicleId114
2020-08-02 23:49:46,369 -46 - relationCalculateWithRange - DEBUG - reservedIndexes.__len__()13
2020-08-02 23:49:46,370 -47 - relationCalculateWithRange - DEBUG - propertyTensors[targetVehicleId].shape[0]6
2020-08-02 23:49:46,383 -45 - relationCalculateWithRange - DEBUG - targetVehicleId115
2020-08-02 23:49:46,384 -46 - relationCalculateWithRange - DEBUG - reservedIndexes.__len__()11
2020-08-02 23:49:46,384 -47 - relationCalculateWithRange - DEBUG - propertyTensors[targetVehicleId].shape[0]6
2020-08-02 23:49:46,398 -45 - relationCalculateWithRange - DEBUG - targetVehicleId116
2020-08-02 23:49:46,399 -46 - relationCalculateWithRange - DEBUG - reservedIndexes.__len__()9
2020-08-02 23:49:46,399 -47 - rela

...

2020-08-02 23:50:16,101 -47 - relationCalculateWithRange - DEBUG - propertyTensors[targetVehicleId].shape[0]6
2020-08-02 23:50:16,114 -45 - relationCalculateWithRange - DEBUG - targetVehicleId115
2020-08-02 23:50:16,114 -46 - relationCalculateWithRange - DEBUG - reservedIndexes.__len__()11
2020-08-02 23:50:16,115 -47 - relationCalculateWithRange - DEBUG - propertyTensors[targetVehicleId].shape[0]6
2020-08-02 23:50:16,128 -45 - relationCalculateWithRange - DEBUG - targetVehicleId116
2020-08-02 23:50:16,128 -46 - relationCalculateWithRange - DEBUG - reservedIndexes.__len__()10
2020-08-02 23:50:16,129 -47 - relationCalculateWithRange - DEBUG - propertyTensors[targetVehicleId].shape[0]6
2020-08-02 23:50:16,143 -45 - relationCalculateWithRange - DEBUG - targetVehicleId117
2020-08-02 23:50:16,144 -46 - relationCalculateWithRange - DEBUG - reservedIndexes.__len__()20
2020-08-02 23:50:16,144 -47 - relationCalculateWithRange - DEBUG - propertyTensors[targetVehicleId].shape[0]6
2020-08-02 23:50:

2020-08-02 23:50:16,525 -45 - relationCalculateWithRange - DEBUG - targetVehicleId143
2020-08-02 23:50:16,526 -46 - relationCalculateWithRange - DEBUG - reservedIndexes.__len__()20
2020-08-02 23:50:16,526 -47 - relationCalculateWithRange - DEBUG - propertyTensors[targetVehicleId].shape[0]6
2020-08-02 23:50:16,540 -45 - relationCalculateWithRange - DEBUG - targetVehicleId144
2020-08-02 23:50:16,541 -46 - relationCalculateWithRange - DEBUG - reservedIndexes.__len__()20
2020-08-02 23:50:16,541 -47 - relationCalculateWithRange - DEBUG - propertyTensors[targetVehicleId].shape[0]6
2020-08-02 23:50:16,555 -45 - relationCalculateWithRange - DEBUG - targetVehicleId145
2020-08-02 23:50:16,555 -46 - relationCalculateWithRange - DEBUG - reservedIndexes.__len__()20
2020-08-02 23:50:16,556 -47 - relationCalculateWithRange - DEBUG - propertyTensors[targetVehicleId].shape[0]6
2020-08-02 23:50:16,570 -45 - relationCalculateWithRange - DEBUG - targetVehicleId146
2020-08-02 23:50:16,570 -46 - relationCal

2020-08-02 23:50:16,947 -46 - relationCalculateWithRange - DEBUG - reservedIndexes.__len__()20
2020-08-02 23:50:16,947 -47 - relationCalculateWithRange - DEBUG - propertyTensors[targetVehicleId].shape[0]6
2020-08-02 23:50:16,961 -45 - relationCalculateWithRange - DEBUG - targetVehicleId172
2020-08-02 23:50:16,961 -46 - relationCalculateWithRange - DEBUG - reservedIndexes.__len__()20
2020-08-02 23:50:16,962 -47 - relationCalculateWithRange - DEBUG - propertyTensors[targetVehicleId].shape[0]6
2020-08-02 23:50:16,976 -45 - relationCalculateWithRange - DEBUG - targetVehicleId173
2020-08-02 23:50:16,976 -46 - relationCalculateWithRange - DEBUG - reservedIndexes.__len__()20
2020-08-02 23:50:16,976 -47 - relationCalculateWithRange - DEBUG - propertyTensors[targetVehicleId].shape[0]6
2020-08-02 23:50:16,990 -45 - relationCalculateWithRange - DEBUG - targetVehicleId174
2020-08-02 23:50:16,991 -46 - relationCalculateWithRange - DEBUG - reservedIndexes.__len__()20
2020-08-02 23:50:16,991 -47 - re

2020-08-02 23:50:17,363 -47 - relationCalculateWithRange - DEBUG - propertyTensors[targetVehicleId].shape[0]6
2020-08-02 23:50:17,377 -45 - relationCalculateWithRange - DEBUG - targetVehicleId200
2020-08-02 23:50:17,377 -46 - relationCalculateWithRange - DEBUG - reservedIndexes.__len__()20
2020-08-02 23:50:17,378 -47 - relationCalculateWithRange - DEBUG - propertyTensors[targetVehicleId].shape[0]6
2020-08-02 23:50:17,391 -45 - relationCalculateWithRange - DEBUG - targetVehicleId201
2020-08-02 23:50:17,392 -46 - relationCalculateWithRange - DEBUG - reservedIndexes.__len__()20
2020-08-02 23:50:17,392 -47 - relationCalculateWithRange - DEBUG - propertyTensors[targetVehicleId].shape[0]6
2020-08-02 23:50:17,406 -45 - relationCalculateWithRange - DEBUG - targetVehicleId202
2020-08-02 23:50:17,406 -46 - relationCalculateWithRange - DEBUG - reservedIndexes.__len__()20
2020-08-02 23:50:17,406 -47 - relationCalculateWithRange - DEBUG - propertyTensors[targetVehicleId].shape[0]6
2020-08-02 23:50:

2020-08-02 23:50:17,789 -45 - relationCalculateWithRange - DEBUG - targetVehicleId228
2020-08-02 23:50:17,789 -46 - relationCalculateWithRange - DEBUG - reservedIndexes.__len__()20
2020-08-02 23:50:17,789 -47 - relationCalculateWithRange - DEBUG - propertyTensors[targetVehicleId].shape[0]6
2020-08-02 23:50:17,803 -45 - relationCalculateWithRange - DEBUG - targetVehicleId229
2020-08-02 23:50:17,803 -46 - relationCalculateWithRange - DEBUG - reservedIndexes.__len__()20
2020-08-02 23:50:17,804 -47 - relationCalculateWithRange - DEBUG - propertyTensors[targetVehicleId].shape[0]6
2020-08-02 23:50:17,817 -45 - relationCalculateWithRange - DEBUG - targetVehicleId230
2020-08-02 23:50:17,818 -46 - relationCalculateWithRange - DEBUG - reservedIndexes.__len__()20
2020-08-02 23:50:17,818 -47 - relationCalculateWithRange - DEBUG - propertyTensors[targetVehicleId].shape[0]6
2020-08-02 23:50:17,831 -45 - relationCalculateWithRange - DEBUG - targetVehicleId231
2020-08-02 23:50:17,832 -46 - relationCal

In [84]:
logging.debug(fromAllToStr("relationInAllTimeStep.shape:",relationInAllTimeStep.shape))
logging.debug(fromAllToStr("discountInAllTimeStep,shape:",discountInAllTimeStep.sha))

2020-08-02 23:51:22,962 -1 - <module> - DEBUG - relationInAllTimeStep.shape:torch.Size([10, 5000, 12])
2020-08-02 23:51:22,964 -2 - <module> - DEBUG - discountInAllTimeStep,shape:tensor([[1.0000, 0.8567, 0.7661,  ..., 0.0000, 0.0000, 0.0000],
        [1.0000, 0.8584, 0.7662,  ..., 0.0000, 0.0000, 0.0000],
        [1.0000, 0.8613, 0.7676,  ..., 0.0000, 0.0000, 0.0000],
        ...,
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]])


In [47]:
loaderIter=iter(datasetV3Loader)
toTestDifferenceItem, toTestDifferenceLabel=loaderIter.__next__()
logging.debug(fromAllToStr("toTestDifferenceItem[0]:",toTestDifferenceItem[0][0][0]))
differenceSeries=differenceBetweenTwoFrameForBatch(toTestDifferenceItem)
logging.debug(fromAllToStr('differenceSeries.shape:',differenceSeries.shape,'toTestDifferenceItem.shape:',\
                          toTestDifferenceItem.shape))

2020-08-02 22:52:35,562 -3 - <module> - DEBUG - toTestDifferenceItem[0]:tensor([0.2609, 0.3732, 0.2600, 0.1253, 0.2766, 0.4081, 0.2603, 0.0877, 0.1010,
        0.3945, 0.0981, 0.2555, 0.4153, 0.0874, 0.5983, 0.3963, 0.5847, 0.2504,
        0.7671, 0.1082, 0.4188, 0.7289, 0.2566, 0.7379, 0.3865, 0.5530, 0.0989,
        0.2716, 0.5814, 0.0974, 0.2384, 0.4232, 0.5597, 0.7003, 0.2451, 0.3901,
        0.5692, 0.7398, 0.7114, 0.1005, 0.4156, 0.7429, 0.1141, 0.2707, 0.4059,
        0.5494, 0.5665, 0.6981, 0.2740, 0.5297, 0.2423, 0.2305, 0.6526, 0.4134,
        0.5797, 0.0745, 0.4274, 0.7192, 0.5536, 0.2522, 0.0932, 0.4223, 0.7164,
        0.2602, 0.0877, 0.5855, 0.7187, 0.0980, 0.5927, 0.3995, 0.2363, 0.7187,
        0.0515, 0.4247, 0.1640, 0.2656, 0.5564, 0.7128, 0.2588, 0.4174, 0.5691,
        0.1410, 0.1012, 0.7484, 0.2568, 0.4133, 0.5571, 0.7290, 0.2833, 0.0737,
        0.5728, 0.7163, 0.4280, 0.5840, 0.2535, 0.0793, 0.4096, 0.7199, 0.2609,
        0.5589, 0.5537, 0.3944, 0.1324, 0.2456, 

2020-08-02 22:52:35,598 -15 - differenceBetweenTwoFrameForBatch - DEBUG - tensor([[[[0.2609, 0.3732, 0.2600,  ..., 0.0000, 0.0000, 0.0000],
          [0.9271, 0.9241, 0.8958,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.2065, 0.2732, 0.1412,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.7759, 1.0000, 1.0000,  ..., 0.5000, 0.5000, 0.5000]],

         [[0.2606, 0.3778, 0.2615,  ..., 0.0000, 0.0000, 0.0000],
          [0.9280, 0.9253, 0.8964,  ..., 0.0000, 0.0000, 0.0000],
          [0.0020, 0.0038, 0.0015,  ..., 0.0000, 0.0000, 0.0000],
          [0.2131, 0.2852, 0.1586,  ..., 0.0000, 0.0000, 0.0000],
          [0.0072, 0.0124, 0.0096,  ..., 0.0000, 0.0000, 0.0000],
          [0.7759, 0.9178, 1.0000,  ..., 0.5000, 0.5000, 0.5000]],

         [[0.2600, 0.3839, 0.2639,  ..., 0.0000, 0.0000, 0.0000],
          [0.9289, 0.9266, 0.8971,  ..., 0.0000, 0.0000, 0.0000]

2020-08-02 22:52:35,614 -16 - differenceBetweenTwoFrameForBatch - DEBUG - tensor([[[[0.2609, 0.3732, 0.2600,  ..., 0.0000, 0.0000, 0.0000],
          [0.9271, 0.9241, 0.8958,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.2065, 0.2732, 0.1412,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.7759, 1.0000, 1.0000,  ..., 0.5000, 0.5000, 0.5000]],

         [[0.2606, 0.3778, 0.2615,  ..., 0.0000, 0.0000, 0.0000],
          [0.9280, 0.9253, 0.8964,  ..., 0.0000, 0.0000, 0.0000],
          [0.0020, 0.0038, 0.0015,  ..., 0.0000, 0.0000, 0.0000],
          [0.2131, 0.2852, 0.1586,  ..., 0.0000, 0.0000, 0.0000],
          [0.0072, 0.0124, 0.0096,  ..., 0.0000, 0.0000, 0.0000],
          [0.7759, 0.9178, 1.0000,  ..., 0.5000, 0.5000, 0.5000]],

         [[0.2600, 0.3839, 0.2639,  ..., 0.0000, 0.0000, 0.0000],
          [0.9289, 0.9266, 0.8971,  ..., 0.0000, 0.0000, 0.0000]

2020-08-02 22:52:35,615 -6 - <module> - DEBUG - differenceSeries.shape:torch.Size([5, 9, 6, 250])toTestDifferenceItem.shape:torch.Size([5, 10, 6, 250])


In [43]:
logging.info('test')
logTensor=torch.zeros(3,3)
logging.debug(str(logTensor))

2020-07-21 15:04:42,320 -1 - <module> - INFO - test
2020-07-21 15:04:42,322 -3 - <module> - DEBUG - tensor([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]])


In [44]:
testTensor=torch.rand((6))
logging.debug(testTensor)
logging.debug(testTensor.expand(3,6))

2020-07-21 15:04:42,328 -2 - <module> - DEBUG - tensor([0.0164, 0.6609, 0.1991, 0.0609, 0.6587, 0.5410])
2020-07-21 15:04:42,330 -3 - <module> - DEBUG - tensor([[0.0164, 0.6609, 0.1991, 0.0609, 0.6587, 0.5410],
        [0.0164, 0.6609, 0.1991, 0.0609, 0.6587, 0.5410],
        [0.0164, 0.6609, 0.1991, 0.0609, 0.6587, 0.5410]])


# test seq2seq relation model

#test code
if runSeq2SeqRelationModel:
    outputTimeFrame=5
    RelationLSTMSeq2SeqModel=RelationLSTMSeq2Seq(outputTimeFrame=outputTimeFrame)
    datasetV3Instance=tensorsDatasetV3(trajectoryFileList,lableTensorEachBatch=outputTimeFrame)
    datasetV3Loader=DataLoader(datasetV3Instance,batch_size=2)
    V3iter=iter(datasetV3Loader)
    

#test code
if runSeq2SeqRelationModel:
    RelationLSTMSeq2SeqModel=RelationLSTMSeq2Seq(outputTimeFrame=outputTimeFrame)
    inputs, labels=V3iter.__next__()
    inputs=fromObjectsToRelationPairsBatchAndTimestepVersion(inputs)
    outputs=RelationLSTMSeq2SeqModel(inputs)
    print(outputs.shape)

In [38]:
#loading model and dataset
if runSeq2SeqRelationModel:
    epochs=500
    itersInEachEpoch=100
    outputTimeFrame=5
    relationLSTMSeq2SeqModel=RelationLSTMSeq2Seq(outputTimeFrame=outputTimeFrame)
    datasetV3Instance=tensorsDatasetV3(trajectoryFileList,lableTensorEachBatch=outputTimeFrame)
    datasetV3Loader=DataLoader(datasetV3Instance,batch_size=5,num_workers=4)
    V3iter=iter(datasetV3Loader)
    if useGpu:
        relationLSTMSeq2SeqModel.cuda()

In [38]:
if runSeq2SeqRelationModel:
    if not isTest:
        learningRate=1e-3
        lossFn=elementWeightedLoss(100,0.1,0.1)
        lambdaWholeNet=lambda epoch: 0.5**(epoch//30)
        optim=torch.optim.RMSprop([{'params':relationLSTMSeq2SeqModel.parameters(),'initial_lr':learningRate}],lr=learningRate)
        lrSchedule=torch.optim.lr_scheduler.LambdaLR(optim,lambdaWholeNet,last_epoch=10)
        relationLSTMSeq2SeqModel.train()

# training seq2seqRelation model

In [45]:
if runSeq2SeqRelationModel:
    if not isTest:
        for epoch in range(epochs):
            for iteration in range(itersInEachEpoch):
                try:
                    inputs, labels=V3iter.__next__()
                except StopIteration:
                    V3iter=iter(datasetV3Loader)
                    inputs, labels=V3iter.__next__()
                inputs=fromObjectsToRelationPairsBatchAndTimestepVersion(inputs)
                labels=labels.permute(0,1,3,2)
                if useGpu:
                    inputs=Variable(inputs.cuda())
                    labels=Variable(labels.cuda())
                outputs=relationLSTMSeq2SeqModel(inputs)
                print('compare the shape of outputs and labels:',outputs.shape, labels.shape)
                loss=lossFn(outputs,labels)
                optim.zero_grad()
                loss.backward()
                optim.step()
                print('loss in opoch ',epoch,',iteration',iteration,':',loss)
            lrSchedule.step()
            if epoch%10==0:
                torch.save(relationLSTMSeq2SeqModel.state_dict(),\
                           'relationLSTMSeq2SeqMode_in_epoch_weightedLoss_'+str(epoch)+'.pt')

# testing seq2seqRelation model

In [51]:
datasetV3Loader=DataLoader(datasetV3Instance,batch_size=2,num_workers=4)
V3iter=iter(datasetV3Loader)

In [64]:
if isTest:
    relationLSTMSeq2SeqModel.eval()
    with torch.no_grad():
        relationLSTMSeq2SeqModel.load_state_dict(torch.load('relationLSTMSeq2SeqMode_in_epoch_weightedLoss_80.pt'))
#         inputs,labels=V3iter.__next__()
        itemIndex=8000
        inputs,labels=datasetV3Instance.__getitem__(itemIndex)
        inputs=inputs.unsqueeze(0)
        labels=labels.unsqueeze(0)
        for ii in range(4):
            newInput, newLabel=datasetV3Instance.__getitem__(itemIndex+ii)
            newInput=newInput.unsqueeze(0)
            newLabel=newLabel.unsqueeze(0)
            inputs=torch.cat((inputs,newInput),0)
            labels=torch.cat((labels,newLabel),0)
            
        print(inputs.shape,labels.shape)

        inputs=fromObjectsToRelationPairsBatchAndTimestepVersion(inputs)
        print(inputs.shape)
        labels=labels.permute(0,1,3,2)
        outputs=relationLSTMSeq2SeqModel(inputs)
        print(outputs.shape)
        normalizationDict=datasetV3Instance.getNormalizationDict()
        for i in range(labels.shape[1]):
            resultImage=visualizeTensorData(outputs[0,i,:,0],outputs[0,i,:,1],normalizationDict=normalizationDict)
            fileName='./predictWithRelationSeq2Seq/'+str(i)+'.png'
            cv2.imwrite(fileName,resultImage)
            resultImage=visualizeTensorData(labels[0,i,:,0],labels[0,i,:,1],normalizationDict=normalizationDict)
            fileName='./predictWithRelationSeq2Seq/'+'l'+str(i)+'.png'
            cv2.imwrite(fileName,resultImage)
        

torch.Size([5, 10, 6, 250]) torch.Size([5, 5, 6, 250])
torch.Size([5, 10, 62250, 12])
in seq2seq, inputs shape: torch.Size([5, 10, 62250, 12])
torch.Size([5, 5, 250, 6])


# test lstm relation model

In [37]:
print('start test lstm relation model')
if runRelationLSTM:
    datasetV3Instance=tensorsDatasetV3(trajectoryFileList)
    datasetV3Loader=DataLoader(datasetV3Instance,batch_size=2)
    V3iter=iter(datasetV3Loader)
    relationLSTMInstance=RelationLSTM()
    if useGpu:
        relationLSTMInstance.cuda()

start test lstm relation model


%matplotlib inline
def calculateXandN(x,n):
    return (1+x)/(n+1+2*x)
ns=[]
xs=[]
for i in range(0,100):
    xs.append(i)
for i in range(0,10):
    ns.append(i)
lines=[]
for i in range(20):
    lines.append([])
for x in xs:
    for n in ns:
        lines[n].append(calculateXandN(x,n))
plt.figure(figsize=(20,20))
plt.subplot(241)
plt.plot(xs,lines[0],'b')
plt.plot(xs,lines[1],'g')
plt.plot(xs,lines[2],'k')
plt.plot(xs,lines[3],'y')
plt.plot(xs,lines[4],'m')
plt.plot(xs,lines[5],'-')
plt.subplot(242)
plt.plot(ns,lines[0],'b')
plt.subplot(243)
plt.plot(ns,lines[1],'b')
plt.subplot(244)
plt.plot(ns,lines[2],'b')
plt.subplot(245)
plt.plot(ns,lines[3],'b')
plt.subplot(246)
plt.plot(ns,lines[4],'b')
plt.subplot(247)
plt.plot(ns,lines[5],'b')

In [33]:
#training relation lstm
if runRelationLSTM:
    if not isTest:
        learningRate=1e-3
        MSELoss=nn.MSELoss()
        lambdaWholeNet=lambda epoch: 0.5**(epoch//30)
        optim=torch.optim.RMSprop([{'params':relationLSTMInstance.parameters(),'initial_lr':learningRate}],lr=learningRate)
        lrSchedule=torch.optim.lr_scheduler.LambdaLR(optim,lambdaWholeNet,last_epoch=10)
        relationLSTMInstance.train()

In [43]:
if runRelationLSTM:
    #training relation lstm
    if not runOnG814:
        %matplotlib inline
    from IPython.display import clear_output
    normalizationDict=datasetV3Instance.getNormalizationDict()
    optim.zero_grad()
    if not isTest:  
        losses=[]
        iterInEpoch=50
        for epoch in range(5):
            print(epoch)
            i=0
            for inputs,label in V3iter:
                i=i+1
                if i>iterInEpoch*(epoch+1):
                    break
        #         print(i)
                inputs=fromObjectsToRelationPairsBatchAndTimestepVersion(inputs)
                print('label.shape',label.shape)
                label=label[:,0,:,:].squeeze()
                label=label.permute(0,2,1)
#                 label=label[:,tupleForEachVehicle,0:6]
                if useGpu:
                    inputs=Variable(inputs.cuda())
                    label=Variable(label.cuda())
                output=relationLSTMInstance(inputs)

        #         print(output[0,0:10,:],secondObjects[0,0:10,:])
                print('output.shape',output.shape)
                loss=MSELoss(output,label)
                if i<5:
                    print('epoch ',epoch, ' i', i,' loss',loss)
        #         print(loss)
                losses.append(loss.item())
                optim.zero_grad()
                loss.backward()
                optim.step()
    #             outputView=output.reshape((7,6,250)).cpu()
    #             for j in range(10):
    #                 inputs=inputs.cpu()
    #                 resultImage=visualizeTensorData(inputs[0,j,0,:],inputs[0,j,1,:],normalizationDict=normalizationDict)
    #                 fileName='./predictWithRelationLSTM/'+str((epoch+1)*10000000+i*100000+j)+'.png'
    #                 cv2.imwrite(fileName,resultImage)
    #             resultImage=visualizeTensorData(outputView[0,0,:],outputView[0,1,:],normalizationDict=normalizationDict)
    #             fileName='./predictWithLSTMOnly/'+str((epoch+1)*10000000+i*100000+50)+'.png' #the predicted image is named with string which last two number is 50(because j < 50)
    #             cv2.imwrite(fileName,resultImage)
            lrSchedule.step()
        plt.figure(figsize=(30,30))
        plt.plot(losses)
        plt.savefig('./losses.png')
        torch.save(relationLSTMInstance.state_dict(),'./relationLSTM.pt')

0
label.shape torch.Size([2, 10, 6, 250])
in step,combined.shape torch.Size([2, 250, 40])


/home/wangyuchen/.conda/envs/pytorch/lib/python3.6/site-packages/torch/nn/functional.py:1351: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/home/wangyuchen/.conda/envs/pytorch/lib/python3.6/site-packages/torch/nn/functional.py:1340: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


in step,combined.shape torch.Size([2, 250, 40])
in step,combined.shape torch.Size([2, 250, 40])
in step,combined.shape torch.Size([2, 250, 40])
in step,combined.shape torch.Size([2, 250, 40])
in step,combined.shape torch.Size([2, 250, 40])
in step,combined.shape torch.Size([2, 250, 40])
in step,combined.shape torch.Size([2, 250, 40])
in step,combined.shape torch.Size([2, 250, 40])
in step,combined.shape torch.Size([2, 250, 40])
torch.Size([2, 10, 62250, 12])
output.shape torch.Size([2, 250, 6])
epoch  0  i 1  loss tensor(0.0382, grad_fn=<MseLossBackward>)
label.shape torch.Size([2, 10, 6, 250])
in step,combined.shape torch.Size([2, 250, 40])
in step,combined.shape torch.Size([2, 250, 40])
in step,combined.shape torch.Size([2, 250, 40])
in step,combined.shape torch.Size([2, 250, 40])
in step,combined.shape torch.Size([2, 250, 40])
in step,combined.shape torch.Size([2, 250, 40])
in step,combined.shape torch.Size([2, 250, 40])
in step,combined.shape torch.Size([2, 250, 40])
in step,combin

KeyboardInterrupt: 

# test relation-object model over a period of time

In [ ]:
if runObjectRelationNet:
    normalizationDict=datasetV2.getNormalizationDict()

    datas=[]
    for i in range(0,6):
        break
        datas.append([])
    for ii in range(0,2000):
        break
        theInput,second=datasetV2.__getitem__(ii)
        theInput=theInput.unsqueeze(0)
        second=second.unsqueeze(0)
        #         print(i)
        #         theInput,second=item
        if useGpu:
            theInput=Variable(theInput.cuda())
            second=Variable(second.cuda())
        inputObjects=theInput[:,tupleForEachVehicle,0:6]
        secondObjects=second[:,tupleForEachVehicle,0:6]
        for i in range(0,6):
            datas[i].append(inputObjects[0,0,i])
    #     print('inputObjects.shape',inputObjects.shape)
    #     resultImage=visualizeTensorData(inputObjects[0,:,0].cpu(),inputObjects[0,:,1].cpu(),normalizationDict=normalizationDict)
    #     fileName='./resultImage/'+str(ii)+'.png'
    #     cv2.imwrite(fileName,resultImage)
    timeStamp=int(time.time())
    dirName='resultImage'+str(timeStamp)
    os.mkdir(dirName)
    if isTest:
        with torch.no_grad():
            wholeNet.eval()
            theInput,second=datasetV2.__getitem__(5000)
            theInput=theInput.unsqueeze(0)
            second=second.unsqueeze(0)
        #         print(i)
    #         theInput,second=item
            if useGpu:
                theInput=Variable(theInput.cuda())
                second=Variable(second.cuda())
            inputObjects=theInput[:,tupleForEachVehicle,0:6]
            secondObjects=second[:,tupleForEachVehicle,0:6]
            print('inputObjects.shape',inputObjects.shape)
            resultImage=visualizeTensorData(inputObjects[0,:,0].cpu(),inputObjects[0,:,1].cpu(),normalizationDict=normalizationDict)
            fileName='./resultImage/'+'0000000000000000'+'.png'
            cv2.imwrite(fileName,resultImage)
            stepInput=fromObjectsToRelationPairs(inputObjects[0].permute(1,0)).unsqueeze(0)
        #             output=wholeNet(theInput)
            print(stepInput.shape)
        #             print(output.shape)
            #predict step by step
            for step in range(500):
                output=wholeNet(stepInput)
                print('step: ',step)
                for ii in range(output.shape[1]):
                    print(output[0,ii])
    #             break
    #             for j in range(10):
    #                 print(stepInput[:,tupleForEachVehicle,0:6][0,j,:])
    #                 print(output[0,j,:])
    #                 print()
                stepInput=fromObjectsToRelationPairs(output[0].permute(1,0)).unsqueeze(0)
    #             print('outputShape',output.shape)
    #             print('outputshape[0]',output[0].shape)
                resultImage=visualizeTensorData(output[0,:,0].cpu(),output[0,:,1].cpu(),normalizationDict=normalizationDict)

                import os
                fileName='./'+dirName+'/'+str(1000000+step)+'.png'
                cv2.imwrite(fileName,resultImage)

# training and testing process for 'fromRelationToObjectnetwork'

In [ ]:
if runObjectRelationNet:
    #generate a tuple in which each element is the index of a vehicle
    #the tuple is used to get the property of each vehicle from the left side of data from the dataset function
    listForEachVehicle=[]
    for i in range(maxMatrixIndex):
        listForEachVehicle.append(i*(maxMatrixIndex-1))
    tupleForEachVehicle=tuple(listForEachVehicle)

    dataloaderV2=DataLoader(datasetV2,batch_size=1,shuffle=True)


    wholeNet=fromRelationToObjectNetwork()
    if isTest:
        wholeNet.load_state_dict(torch.load(modelPath))


    if useGpu:
        wholeNet.cuda()



In [ ]:
if runObjectRelationNet:
    if not isTest:
        learningRate=1e-3
        MSELoss=nn.MSELoss()
        lambdaWholeNet=lambda epoch: 0.5**(epoch//30)
        optim=torch.optim.RMSprop([{'params':wholeNet.parameters(),'initial_lr':learningRate}],lr=learningRate)
        lrSchedule=torch.optim.lr_scheduler.LambdaLR(optim,lambdaWholeNet,last_epoch=10)
        wholeNet.train()

In [ ]:
if runObjectRelationNet:
    if not runOnG814:
        %matplotlib inline
    from IPython.display import clear_output
    if not isTest:  
        losses=[]
        iterInEpoch=50
        for epoch in range(300):
            print(epoch)
            for i,item in enumerate(dataloaderV2):
                if i>iterInEpoch:
                    break
        #         print(i)
                theInput,second=item
                if useGpu:
                    theInput=Variable(theInput.cuda())
                    second=Variable(second.cuda())
                secondObjects=second[:,tupleForEachVehicle,0:6]

                output=wholeNet(theInput)
        #         print(output[0,0:10,:],secondObjects[0,0:10,:])
                loss=MSELoss(output,secondObjects)
                if i<5:
                    print('epoch ',epoch, ' i', i,' loss',loss)
        #         print(loss)
                losses.append(loss.item())
                optim.zero_grad()
                loss.backward()
                optim.step()
            lrSchedule.step()
        plt.figure(figsize=(30,30))
        plt.plot(losses)
        plt.savefig('./losses.png')
        torch.save(wholeNet.state_dict(),'./wholeNet_300epoch_50perEpoch.pt')

# training simple LSTM module

In [ ]:
if runLSTM:
    #lstm version 
    if not isTest:
        learningRate=1e-3
        MSELoss=nn.MSELoss()
        lambdaWholeNet=lambda epoch: 0.5**(epoch//30)
        optim=torch.optim.RMSprop([{'params':lstmModel.parameters(),'initial_lr':learningRate}],lr=learningRate)
        lrSchedule=torch.optim.lr_scheduler.LambdaLR(optim,lambdaWholeNet,last_epoch=10)
        lstmModel.train()

In [ ]:
if runLSTM:
    #lstm version
    if not runOnG814:
        %matplotlib inline
    from IPython.display import clear_output
    normalizationDict=datasetV3.getNormalizationDict()
    if not isTest:  
        losses=[]
        iterInEpoch=50
        for epoch in range(5):
            print(epoch)
            i=0
            for inputs,label in iterV3:
                i=i+1
                if i>iterInEpoch*(epoch+1):
                    break
        #         print(i)
                if useGpu:
                    inputs=Variable(inputs.cuda())
                    label=Variable(label.cuda())
                output=lstmModel(inputs.reshape((inputs.shape[0],inputs.shape[1],-1)))

        #         print(output[0,0:10,:],secondObjects[0,0:10,:])
                loss=MSELoss(output,label.squeeze().reshape((label.shape[0],-1)))
                if i<5:
                    print('epoch ',epoch, ' i', i,' loss',loss)
        #         print(loss)
                losses.append(loss.item())
                optim.zero_grad()
                loss.backward()
                optim.step()
                outputView=output.reshape((7,6,250)).cpu()
                for j in range(10):
                    inputs=inputs.cpu()
                    resultImage=visualizeTensorData(inputs[0,j,0,:],inputs[0,j,1,:],normalizationDict=normalizationDict)
                    fileName='./predictWithLSTMOnly/'+str((epoch+1)*10000000+i*100000+j)+'.png'
                    cv2.imwrite(fileName,resultImage)
                resultImage=visualizeTensorData(outputView[0,0,:],outputView[0,1,:],normalizationDict=normalizationDict)
                fileName='./predictWithLSTMOnly/'+str((epoch+1)*10000000+i*100000+50)+'.png' #the predicted image is named with string which last two number is 50(because j < 50)
                cv2.imwrite(fileName,resultImage)
            lrSchedule.step()
        plt.figure(figsize=(30,30))
        plt.plot(losses)
        plt.savefig('./losses.png')
        torch.save(wholeNet.state_dict(),'./wholeNet_300epoch_50perEpoch.pt')

#draw properties of a single tensor over time
#output results step by step
plt.subplot(321)
plt.plot(datas[0])
plt.subplot(322)
plt.plot(datas[1])
plt.subplot(323)
plt.plot(datas[2])
plt.subplot(324)
plt.plot(datas[3])
plt.subplot(325)
plt.plot(datas[4])
plt.subplot(326)
plt.plot(datas[5])